In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.impute import KNNImputer
import pickle

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# Handling Errant Values 

In [3]:
raw_file = "/home/ec2-user/SageMaker/Team-5/Submission/Datasets/TN/TN_allinone01122.csv"
df = pd.read_csv(raw_file)

/tmp/ipykernel_36581/1749200401.py:2: DtypeWarning: Columns (3,4,5,6,8,11,14,16,17,19,20,23,25,26,27,28,29,30,31,33,35,37,42,48,50,52,54,56,58,60,62,65,68,71,74,77,78,79,80,81,83,84,85,86,88,92,93,94,95,96,97,99,102,105,106,107,109,110,118,131,132,133,134,135,136,137,139,141,144,148,150,152,153,157,158,162,163,167,168,172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(raw_file)


In [4]:
df

,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,_22Jul2009ConsentSignedDay,_22Jul2009ConsentSignedMonth,_22Jul2009ConsentSignedYear,...,CREA,CHL,BIC,ALT,ALP,ALB,UREA,AST,CPEP,INST
0,200023,07/01/2015,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,200071,01/24/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200074,04/23/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,unknown,Unknown,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200082,03/30/2010,PRN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200082,06/13/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503518,941306,06/01/2012,Screening,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503519,941309,12/17/2012,Screening,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503520,941310,03/03/2010,Screening,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503521,941316,03/06/2012,Screening,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503523 entries, 0 to 503522
Columns: 584 entries, MaskID to INST
dtypes: float64(500), int64(1), object(83)
memory usage: 2.2+ GB


In [6]:
## Converting heights
df['Heightin_to_cm'] = df['Heightin'] * 2.53999863
df['Height_cm'] = df['Heightin_to_cm'] + df['Heightcm']
df.insert(df.columns.get_loc('Height_cm') + 1, 'Height_cm_units', 'cm')
df.drop(['Heightin_to_cm', 'Heightcm', 'Heightin'], axis=1, inplace=True)

df['Weightlbs_to_kg'] = df['Weightlbs'] * 0.45359237
df['Weight_kg'] = df['Weightlbs_to_kg'] + df['Weightkg']
df.insert(df.columns.get_loc('Weight_kg') + 1, 'Weight_kg_units', 'kg')
df.drop(['Weightlbs_to_kg', 'Weightkg', 'Weightlbs'], axis=1, inplace=True)

In [7]:
## Converting glucose units and organizing columns
for column in df.columns:
    if 'mmol' in column.lower() or 'mgdl' in column.lower():
        new_column_name = column.replace('mmol', '_unit').replace('mgdl', '').replace('Mmoll', '_unit').replace('MgDL', '').replace('mmoll', '_unit')
        df.rename(columns={column: new_column_name}, inplace=True)
        
columns_to_process = ['AvgRecordedFastingGlucoses', 'AvgAllRecordedGlucoses', 'LowestGlucose', 'HighestGlucose']
df[columns_to_process] = df[columns_to_process].apply(lambda x: x.where(x >= 45, x * 18.018))

columns_to_convert = ['AvgRecordedFastingGlucoses_unit', 'AvgAllRecordedGlucoses_unit', 'LowestGlucose_unitl', 'HighestGlucose_unit']
df[columns_to_convert] = df[columns_to_convert].replace('mmol/L', 'mg/dl')



Hemoglobin_process = ['HemoglobinResults']
df[Hemoglobin_process] = df[Hemoglobin_process].apply(lambda x: x.where(x <= 30, x / 10))

Hematocrit_process = ['HematocritResults']
df[Hematocrit_process] = df[Hematocrit_process].apply(lambda x: x.where(x >= 1, x * 100))


column_to_move1 = 'AvgRecordedFastingGlucoses_unit'
target_position1 = 'AvgRecordedFastingGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move1)
target_index = current_positions.index(target_position1)
new_positions = current_positions[:target_index + 1] + [column_to_move1] + current_positions[target_index + 1:]
df = df[new_positions]

column_to_move2 = 'AvgAllRecordedGlucoses_unit'
target_position2 = 'AvgAllRecordedGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move2)
target_index = current_positions.index(target_position2)
new_positions = current_positions[:target_index + 1] + [column_to_move2] + current_positions[target_index + 1:]
df = df[new_positions]

df.replace('mg/dl', 'mg/dL', inplace=True, regex=True)

df.rename(columns={'LowestGlucose_unitl': 'LowestGlucose_unit'}, inplace=True)
print(df.shape)


(503523, 584)


In [8]:
existing_column_index = df.columns.get_loc('HemoglobinResults')
df.insert(existing_column_index + 1, 'HemoglobinResults_Units', "NaN")
df.loc[df['HemoglobinResults'].notna(), 'HemoglobinResults_Units'] = "mg/dL"

existing_column_index = df.columns.get_loc('HbA1c')
df.insert(existing_column_index + 1, 'HbA1c_Units', "NaN")
df.loc[df['HbA1c'].notna(), 'HbA1c_Units'] = "Hb%"

existing_column_index = df.columns.get_loc('MCHresults')
df.insert(existing_column_index + 1, 'MCHresults_Units', "NaN")
df.loc[df['MCHresults'].notna(), 'MCHresults_Units'] = "pg"

existing_column_index = df.columns.get_loc('MCHCresults')
df.insert(existing_column_index + 1, 'MCHCresults_Units', "NaN")
df.loc[df['MCHCresults'].notna(), 'MCHCresults_Units'] = "g/dL"

existing_column_index = df.columns.get_loc('EosinophilsResults')
df.insert(existing_column_index + 1, 'EosinophilsResults_Units', "NaN")
df.loc[df['EosinophilsResults'].notna(), 'EosinophilsResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('BasophilsResults')
df.insert(existing_column_index + 1, 'BasophilsResults_Units', "NaN")
df.loc[df['BasophilsResults'].notna(), 'BasophilsResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('LymphocytesResults')
df.insert(existing_column_index + 1, 'LymphocytesResults_Units', "NaN")
df.loc[df['LymphocytesResults'].notna(), 'LymphocytesResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('HematocritResults')
df.insert(existing_column_index + 1, 'HematocritResults_Units', "NaN")
df.loc[df['HematocritResults'].notna(), 'HematocritResults_Units'] = "% of blood"

existing_column_index = df.columns.get_loc('MCVresults')
df.insert(existing_column_index + 1, 'MCVresults_Units', "NaN")
df.loc[df['MCVresults'].notna(), 'MCVresults_Units'] = "fL"

existing_column_index = df.columns.get_loc('MonocytesResults')
df.insert(existing_column_index + 1, 'MonocytesResults_Units', "NaN")
df.loc[df['MonocytesResults'].notna(), 'MonocytesResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('NeutrophilsResults')
df.insert(existing_column_index + 1, 'NeutrophilsResults_Units', "NaN")
df.loc[df['NeutrophilsResults'].notna(), 'NeutrophilsResults_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('WBCresults')
df.insert(existing_column_index + 1, 'WBCresults_Units', "NaN")
df.loc[df['WBCresults'].notna(), 'WBCresults_Units'] = "x10^9 cells/L"

existing_column_index = df.columns.get_loc('Rbcresults')
df.insert(existing_column_index + 1, 'Rbcresults_Units', "NaN")
df.loc[df['Rbcresults'].notna(), 'Rbcresults_Units'] = "x10^12 cells/L"

existing_column_index = df.columns.get_loc('EosinophilsResultsOR')
df.insert(existing_column_index + 1, 'EosinophilsResultsOR_Units', "NaN")
df.loc[df['EosinophilsResultsOR'].notna(), 'EosinophilsResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('LymphocytesResultsOR')
df.insert(existing_column_index + 1, 'LymphocytesResultsOR_Units', "NaN")
df.loc[df['LymphocytesResultsOR'].notna(), 'LymphocytesResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('BasophilsResultsOR')
df.insert(existing_column_index + 1, 'BasophilsResultsOR_Units', "NaN")
df.loc[df['BasophilsResultsOR'].notna(), 'BasophilsResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('MonocytesResultsOR')
df.insert(existing_column_index + 1, 'MonocytesResultsOR_Units', "NaN")
df.loc[df['MonocytesResultsOR'].notna(), 'MonocytesResultsOR_Units'] = "% of WBC"

existing_column_index = df.columns.get_loc('NeutrophilsResultsOR')
df.insert(existing_column_index + 1, 'NeutrophilsResultsOR_Units', "NaN")
df.loc[df['NeutrophilsResultsOR'].notna(), 'NeutrophilsResultsOR_Units'] = "% of WBC"

In [9]:
columns_to_filter = ['Rbcresults', 'HemoglobinResults',
                  'HematocritResults', 'MCVresults',
                  'PlateletCountResultsNormal', 'MCHresults',
                  'MCHCresults', 'WBCresults',
                  'NeutrophilsResultsOR', 'LymphocytesResultsOR',
                  'MonocytesResultsOR', 'EosinophilsResultsOR',
                  'BasophilsResultsOR']
filter_columns = df.columns[df.columns.str.contains('ResultsNormal', case=False)].tolist()
element_to_remove = 'PlateletCountResultsNormal'
filter_columns.remove(element_to_remove)

print(columns_to_filter)
print(filter_columns)

for col, filter_col in zip(columns_to_filter, filter_columns):
    filter_mask = df[filter_col] == 'Yes'
    lower_bound = df[col].quantile(0.0025)
    upper_bound = df[col].quantile(0.9975)
     # Apply np.where only to the relevant subset
    df.loc[~filter_mask, col] = np.where((df.loc[~filter_mask, col] < lower_bound) | 
                                         (df.loc[~filter_mask, col] > upper_bound), 
                                         np.nan, 
                                         df.loc[~filter_mask, col])
print(df)

['Rbcresults', 'HemoglobinResults', 'HematocritResults', 'MCVresults', 'PlateletCountResultsNormal', 'MCHresults', 'MCHCresults', 'WBCresults', 'NeutrophilsResultsOR', 'LymphocytesResultsOR', 'MonocytesResultsOR', 'EosinophilsResultsOR', 'BasophilsResultsOR']
['RBCResultsNormal', 'HemoglobinResultsNormal', 'HematocritResultsNormal', 'MCVresultsNormal', 'PlateletCountResultsNormal2', 'MCHresultsNormal', 'MCHCresultsNormal', 'WBCresultsNormal', 'NeutrophilsResultsNormal', 'LymphocytesResultsNormal', 'MonocytesResultsNormal', 'EosinophilsResultsNormal', 'BasophilsResultsNormal']
        MaskID    Visit_Dt      Visit _22Jul2009VisitType  \
0       200023  07/01/2015        PRN                 NaN   
1       200071  01/24/2018        PRN                 NaN   
2       200074  04/23/2012        PRN                 NaN   
3       200082  03/30/2010        PRN                 NaN   
4       200082  06/13/2012        PRN                 NaN   
...        ...         ...        ...              

In [10]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503523 entries, 0 to 503522
Columns: 602 entries, MaskID to Weight_kg_units
dtypes: float64(498), int64(1), object(103)
memory usage: 2.3+ GB
None


,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,_22Jul2009ConsentSignedDay,_22Jul2009ConsentSignedMonth,_22Jul2009ConsentSignedYear,...,ALP,ALB,UREA,AST,CPEP,INST,Height_cm,Height_cm_units,Weight_kg,Weight_kg_units
0,200023,07/01/2015,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg
1,200071,01/24/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,kg
2,200074,04/23/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,unknown,Unknown,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,kg
3,200082,03/30/2010,PRN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,kg
4,200082,06/13/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,cm,NaN,kg


In [11]:
# Replace both "." and " ." with NaN
df = df.replace({'.': np.nan, ' .': np.nan, '  .':np.nan})

# Date column harmonization

# 1. Columns of Date

#### Steps of date column harmonization:
1. Filter out date column sets based on their headers
2. Create one date column to replace one set of date columns.
3. assign values in the format of date_time to the new created column.

In [12]:
merged_df=df
#This is a tool to find columns based on names of their headers and check their values. 
contain_list_0 = ['SerumKetonesDay'] 
# Example strings to exclude
not_contain_list_0 = []     

# Converting each string in the lists to lower case
contain_list = [string.lower() for string in contain_list_0]
not_contain_list = [string.lower() for string in not_contain_list_0]

# Filtering columns with case-insensitive comparison
filtered_columns = [col for col in merged_df.columns if 
                    any(contain in col.lower() for contain in contain_list) and 
                    all(not_contain not in col.lower() for not_contain in not_contain_list)]

print("Found ", len(filtered_columns), "columns: ", filtered_columns)

filtered_df = merged_df[filtered_columns].dropna(how='all')

# Output the filtered DataFrame
print(filtered_df.info())

# Uncomment if you want to print top 20 values for each column
for col in filtered_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_df[col].value_counts().nlargest(20))
    print()

Found  0 columns:  []
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame
None


## 1.1 Day, Month, Year

In [13]:
# Filter all columns with "Day", "Month", "Year" as date sets
# in lower case for case-insensitive comparison
# Example strings to include
contain_list_1 = ['Day', "Month", 'Year'] 
# Example strings to exclude
not_contain_list_1 = ['LastYear', 'Last6Months', '3Days', 'Birth', 'MostRecentHbA1c', 'PastYear', 
                      'ChangeInDrugStatusEffective', 'ParticipantDateDispensed', 'MechanisticSampleCollection',
                     'Polydipsia', 'Polyuria', '10days', 'ADHPsoriasisDateofDiagnosis', 'CeliacSprueGlutenDiagnosis',
                     'Datethebloodsamplewasdrawn', 'MHInflammatoryBowelDisease', 'ADHAlopeciaDateofDiagnosis', 
                      'ADHOtherDateofDiagnosis', 'ADHVitiligoDateofDiagnosis', 'VitiligoDiagnosis', 'NextYear',
                      'Polyphagia', 'Fatigue', 'monthly', 'Ketoacidosis', 'UnexplainedWeightLoss', 'HbA1crecordDate']     

# Converting each string in the lists to lower case
contain_list_1_low = [string.lower() for string in contain_list_1]
not_contain_list_1_low = [string.lower() for string in not_contain_list_1]

# Filtering columns with case-insensitive comparison
filtered_date_columns_1 = [col for col in df.columns if 
                    any(contain in col.lower() for contain in contain_list_1_low) and 
                    all(not_contain not in col.lower() for not_contain in not_contain_list_1_low)]

print("Found ", len(filtered_date_columns_1), "columns: ", filtered_date_columns_1)

filtered_df = df[filtered_date_columns_1]

# Output the filtered DataFrame
print(filtered_df.info())

# Uncomment if you want to print top 20 values for each column
#for col in filtered_columns:
#    print(f"Top 20 values for column {col}:")
#    print(df[col].value_counts().nlargest(20))
#    print()

Found  75 columns:  ['_22Jul2009ConsentSignedDay', '_22Jul2009ConsentSignedMonth', '_22Jul2009ConsentSignedYear', '_15Aug2011ConsentDateDay', '_15Aug2011ConsentDateMonth', '_15Aug2011ConsentDateYear', 'Fall2017ConsentDateDay', 'Fall2017ConsentDateMonth', 'Fall2017ConsentDateYear', 'Spring2019ConsentDateMonth', 'Spring2019ConsentDateDay', 'Spring2019ConsentDateYear', 'DateOfBloodDrawDay', 'DateOfBloodDrawMonth', 'DateOfBloodDrawYear', 'DateReportedDay', 'DateReportedMonth', 'DateReportedYear', 'ScreeningDay', 'ScreeningMonth', 'ScreeningYear', 'ConsentSignedDay', 'ConsentSignedMonth', 'ConsentSignedYear', 'DateConsentSignedDay', 'DateConsentSignedMonth', 'DateConsentSignedYear', 'SamplesCollectionDay', 'SamplesCollectionMonth', 'SamplesCollectionYear', 'DateStatusChangeDay', 'DateStatusChangeMonth', 'DateStatusChangeYear', 'DoRDay', 'DoRMonth', 'DoRYear', 'DoWDay', 'DoWMonth', 'DoWYear', 'DiagnosisDay', 'DiagnosisMonth', 'DiagnosisYear', 'InsulinTreatmentStartedDay', 'InsulinTreatmentSt

#### Check month and year columns based on day columns

In [15]:
#check columns for the day
day_column_list_1 = [col for col in filtered_date_columns_1 if "Day" in col]
print("No. of the day columes to be merged: ", len(day_column_list_1))

#create columns for the month and the year based on columns of the day
month_column_list_1 = [name.replace("Day", "Month") for name in day_column_list_1]
year_column_list_1 = [name.replace("Day", "Year") for name in day_column_list_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_month_columns = [name for name in month_column_list_1 if name not in df.columns]
print("No. of month columes in wrong naming format:", len(unmatched_month_columns))
print("Month columes in wrong naming format:", unmatched_month_columns)

unmatched_year_columns = [name for name in year_column_list_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

No. of the day columes to be merged:  40
No. of month columes in wrong naming format: 5
Month columes in wrong naming format: ['AlopeciaDateofDiagnosisMonth', 'LupusDateofDiagnosisMonth', 'PsoriasisDateofDiagnosisMonth', 'VitiligoDateofDiagnosisMonth', 'ADHLupusDateofDiagnosisMonth']
No. of year columes in wrong naming format:  0
Year columes in wrong naming format:  []


#### Check day and year columns based on month columns

In [16]:
#check columns for the day
month_column_list_1_1 = [col for col in filtered_date_columns_1 if "Month" in col]
print("No. of the day columes to be merged: ", len(month_column_list_1_1))

#create columns for the month and the year based on columns of the day
day_column_list_1_1 = [name.replace("Month", "Day") for name in month_column_list_1_1]
year_column_list_1_1 = [name.replace("Month", "Year") for name in month_column_list_1_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_day_columns = [name for name in day_column_list_1_1 if name not in df.columns]
print("No. of day columes in wrong naming format:", len(unmatched_day_columns))
print("Day columes in wrong naming format:", unmatched_day_columns)

unmatched_year_columns = [name for name in year_column_list_1_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

No. of the day columes to be merged:  35
No. of day columes in wrong naming format: 0
Day columes in wrong naming format: []
No. of year columes in wrong naming format:  0
Year columes in wrong naming format:  []


In [17]:
import numpy as np
# Convert 'NA' strings to np.nan
def convert_na(value):
    if pd.isna(value) or value == 'NA':  # using pandas function to check for NA
        return np.nan   # converting to numpy NaN
    else:
        return value

# Function to convert month from string or number to numeric month, handling missing values
def month_to_num(month):
    if pd.isna(month):
        return np.nan
    try:
        # Handle string representation
        if isinstance(month, str):
            datetime_object = pd.to_datetime(month, format='%b')
            return datetime_object.month
        # Handle numeric representation
        elif isinstance(month, (int, float)):
            return int(month)
    except ValueError:
        return np.nan
    
# Create a date column and assign values by combine date columns with similar header
def create_date_column(df, year_col, original_cols):
    parts = year_col.split('Year')
    base_col_name = parts[0]
    suffix = parts[1] if len(parts) > 1 else ''

    day_col = base_col_name + 'Day' + suffix
    month_col = base_col_name + 'Month' + suffix

    if day_col in original_cols and month_col in original_cols:
        date_col = base_col_name + 'Date' + suffix
        df[date_col] = pd.NaT  # Initialize column with NaT (Not-a-Time) for datetime objects
        problematic_rows = []

        for index, row in df.iterrows():
            try:
                if pd.isna(row[year_col]) or pd.isna(row[month_col]) or pd.isna(row[day_col]):
                    continue  # NaNs are already handled by NaT initialization
                else:
                    year = int(row[year_col])
                    month = int(month_to_num(row[month_col]))
                    day = int(row[day_col])
                    df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
            except (ValueError, OverflowError) as e:
                problematic_rows.append((index, row[year_col], row[month_col], row[day_col]))
                df.at[index, date_col] = pd.NaT

        if problematic_rows:
            print("Problematic rows:")
            for row in problematic_rows:
                print(row)
    else:
        print(f"Matching columns for '{year_col}' not found.")

In [18]:
# Convert 'NA' strings to np.nan
for col in filtered_date_columns_1:
    df.loc[:, col] = df[col].apply(convert_na)

# Iterating over columns and creating date columns
for col in year_column_list_1:
    create_date_column(df, col, filtered_date_columns_1)

print(df.info())
df.head()

/tmp/ipykernel_29752/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2012-02-01' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
/tmp/ipykernel_29752/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2012-06-13' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
/tmp/ipykernel_29752/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2018-06-13' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at

/tmp/ipykernel_29752/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-05-06' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
/tmp/ipykernel_29752/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2015-10-11' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
/tmp/ipykernel_29752/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2015-01-29' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at

Matching columns for 'AlopeciaDateofDiagnosisYear' not found.
Matching columns for 'LupusDateofDiagnosisYear' not found.
Matching columns for 'PsoriasisDateofDiagnosisYear' not found.
Matching columns for 'VitiligoDateofDiagnosisYear' not found.
Matching columns for 'ADHLupusDateofDiagnosisYear' not found.


/tmp/ipykernel_29752/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-05-31' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
/tmp/ipykernel_29752/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-05-02' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce').date()
/tmp/ipykernel_29752/1799681261.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-06-13' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.at

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 1102 entries, MaskID to ContactDateDate
dtypes: float64(496), int64(1), object(605)
memory usage: 1.6+ GB
None


,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,_22Jul2009ConsentSignedDay,_22Jul2009ConsentSignedMonth,_22Jul2009ConsentSignedYear,...,LastHbA1cDate,InformedConsentDate,DateProtocolDeviationDate,WrittenInformedConsentDate,Type1DiabetesDiagnosisDate,FluVaccinationDateDate,DateTestsRunDate,StudyDrugReturnedDate,StudyDrugDispensedDate,ContactDateDate
0,200082,03/30/2010,PRN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,200082,06/13/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,200185,07/06/2016,PRN,NaN,Initial Screening Visit,NaN,Yes,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,200273,05/16/2012,PRN,NaN,Initial Screening Visit,Not applicable,Yes,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,200273,06/14/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 1102 entries, MaskID to ContactDateDate
dtypes: float64(496), int64(1), object(605)
memory usage: 1.6+ GB


In [20]:
columns_to_drop = [col for col in filtered_date_columns_1]

print(len(columns_to_drop))
df.drop(columns=columns_to_drop, inplace=True)
df.info()

115
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 987 entries, MaskID to ContactDateDate
dtypes: float64(424), int64(1), object(562)
memory usage: 1.4+ GB


## 1.2 Month, Year

In [21]:
column_pairs = [
    ('BirthMonth', 'BirthYear'),
    ('DateOfBirthMonth', 'DateOfBirthYear'),
    ('HbA1crecordDateMonth', 'HbA1crecordDateYear')
]

In [22]:
# Modified convert_to_datetime function to handle missing columns
def convert_to_datetime(row, month_col, year_col):
    try:
        if pd.notna(row[month_col]) and pd.notna(row[year_col]):
            month = row[month_col].capitalize()  # Capitalize to match standard abbreviations
            year = int(row[year_col])
            return pd.to_datetime(f"{month} {year}", format='%b %Y').strftime('%b %Y')
        else:
            return np.nan
    except (ValueError, TypeError):
        return np.nan
    
def safe_convert_to_datetime(df, month_col, year_col):
    if month_col in df.columns and year_col in df.columns:
        return df.apply(lambda row: convert_to_datetime(row, month_col, year_col), axis=1)
    else:
        return pd.Series([np.nan] * len(df))

# Create a new DataFrame for the new columns
new_columns_df = pd.DataFrame()

In [23]:
# Iterate over each pair and create a new column
for month_col, year_col in column_pairs:
    new_col_name = month_col.replace('Month', '') + 'Date'
    new_columns_df[new_col_name] = safe_convert_to_datetime(df, month_col, year_col)

# Check for missing columns and print them
for month_col, year_col in column_pairs:
    if month_col not in df.columns or year_col not in df.columns:
        print(f"Column pair missing: {month_col}, {year_col}")

# Concatenate the new columns with the original DataFrame
df = pd.concat([df, new_columns_df], axis=1)

# Check the result
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 990 entries, MaskID to HbA1crecordDateDate
dtypes: float64(424), int64(1), object(565)
memory usage: 1.4+ GB
None


,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,WhyAccessingParticipantRecord,ProtocolVersion,...,WrittenInformedConsentDate,Type1DiabetesDiagnosisDate,FluVaccinationDateDate,DateTestsRunDate,StudyDrugReturnedDate,StudyDrugDispensedDate,ContactDateDate,BirthDate,DateOfBirthDate,HbA1crecordDateDate
0,200082,03/30/2010,PRN,NaN,NaN,NaN,NaN,NaN,To enter or view data from a visit which occur...,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
1,200082,06/13/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
2,200185,07/06/2016,PRN,NaN,Initial Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
3,200273,05/16/2012,PRN,NaN,Initial Screening Visit,Not applicable,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
4,200273,06/14/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,3.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN


In [24]:
#df = df.drop(['MostRecentHbA1cDate'], axis=1)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 990 entries, MaskID to HbA1crecordDateDate
dtypes: float64(424), int64(1), object(565)
memory usage: 1.4+ GB


In [26]:
columns_to_drop = [col for pair in column_pairs for col in pair]

# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 984 entries, MaskID to HbA1crecordDateDate
dtypes: float64(421), int64(1), object(562)
memory usage: 1.4+ GB


## 1.3 DD, MM, YYYY

In [27]:
# Filter all columns with "DD", "MM", or "YYYY"
contain_list_1 = ["DD", "MM", "YYYY"] 
# Example strings to exclude
not_contain_list_1 = ['Autoimmuned', 'MMTT', 'TTT', 'TimeOfCollection', 'TimeResultsProcessedByLab']     

# Converting each string in the lists to lower case
#contain_list_1_low = [string.lower() for string in contain_list_1]
#not_contain_list_1_low = [string.lower() for string in not_contain_list_1]

# Filtering columns with case-insensitive comparison
filtered_date_columns_1 = [col for col in df.columns if 
                    any(contain in col for contain in contain_list_1) and 
                    all(not_contain not in col for not_contain in not_contain_list_1)]

print("Found ", len(filtered_date_columns_1), "columns: ", filtered_date_columns_1)

filtered_df = df[filtered_date_columns_1]

# Output the filtered DataFrame
print(filtered_df.info())

Found  18 columns:  ['DateofAssessmentDD', 'DateofAssessmentMM', 'DateofAssessmentYYYY', 'DateCompletionDD', 'DateCompletionMM', 'DateCompletionYYYY', 'F2ClinicalDiagnosisDateDD', 'F2ClinicalDiagnosisDateMM', 'F2ClinicalDiagnosisDateYYYY', 'DateRecordedDD1', 'DateRecordedMM1', 'DateRecordedYYYY1', 'DateRecordedDD2', 'DateRecordedMM2', 'DateRecordedYYYY2', 'DateRecordedDD3', 'DateRecordedMM3', 'DateRecordedYYYY3']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   DateofAssessmentDD           283 non-null    float64
 1   DateofAssessmentMM           283 non-null    object 
 2   DateofAssessmentYYYY         283 non-null    float64
 3   DateCompletionDD             782 non-null    float64
 4   DateCompletionMM             782 non-null    object 
 5   DateCompletionYYYY           782 non-null    float64
 6   F2C

In [28]:
#check columns for the day
day_column_list_1 = [col for col in filtered_date_columns_1 if "DD" in col]
print("No. of the day columes to be merged: ", len(day_column_list_1))

#create columns for the month and the year based on columns of the day
month_column_list_1 = [name.replace("DD", "MM") for name in day_column_list_1]
year_column_list_1 = [name.replace("DD", "YYYY") for name in day_column_list_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_month_columns = [name for name in month_column_list_1 if name not in df.columns]
print("No. of month columes in wrong naming format:", len(unmatched_month_columns))
print("Month columes in wrong naming format:", unmatched_month_columns)

unmatched_year_columns = [name for name in year_column_list_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

No. of the day columes to be merged:  6
No. of month columes in wrong naming format: 0
Month columes in wrong naming format: []
No. of year columes in wrong naming format:  0
Year columes in wrong naming format:  []


In [29]:
#check columns for the day
month_column_list_1_1 = [col for col in filtered_date_columns_1 if "MM" in col]
print("No. of the day columes to be merged: ", len(month_column_list_1_1))

#create columns for the month and the year based on columns of the day
day_column_list_1_1 = [name.replace("MM", "DD") for name in month_column_list_1_1]
year_column_list_1_1 = [name.replace("MM", "YYYY") for name in month_column_list_1_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_day_columns = [name for name in day_column_list_1_1 if name not in df.columns]
print("No. of day columes in wrong naming format:", len(unmatched_day_columns))
print("Day columes in wrong naming format:", unmatched_day_columns)

unmatched_year_columns = [name for name in year_column_list_1_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))
print("Year columes in wrong naming format: ", unmatched_year_columns)

No. of the day columes to be merged:  6
No. of day columes in wrong naming format: 0
Day columes in wrong naming format: []
No. of year columes in wrong naming format:  0
Year columes in wrong naming format:  []


In [30]:
import pandas as pd
import numpy as np

# Function to convert month from string to number, handling missing values
def month_to_num_1(month_str):
    if pd.isna(month_str):
        return np.nan
    try:
        datetime_object = pd.to_datetime(month_str, format='%b')
        return datetime_object.month
    except ValueError:
        return np.nan
    
def create_date_column_2(df, year_col, original_cols):
    parts = year_col.split('YYYY')
    base_col_name = parts[0]
    suffix = parts[1] if len(parts) > 1 else ''

    day_col = base_col_name + 'DD' + suffix
    month_col = base_col_name + 'MM' + suffix

    if day_col in original_cols and month_col in original_cols:
        date_col = base_col_name + 'Date' + suffix
        problematic_rows = []
        for index, row in df.iterrows():
            try:
                if pd.isna(row[year_col]) or pd.isna(row[month_col]) or pd.isna(row[day_col]):
                    df.at[index, date_col] = np.nan
                else:
                    year = int(row[year_col])
                    month = int(month_to_num_1(row[month_col]))
                    day = int(row[day_col])
                    datetime_obj = pd.to_datetime(f"{year}-{month:02d}-{day:02d}")
                    df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
            except (ValueError, OverflowError) as e:
                problematic_rows.append((index, row[year_col], row[month_col], row[day_col]))
                df.at[index, date_col] = np.nan
        if problematic_rows:
            print("Problematic rows:")
            for row in problematic_rows:
                print(row)
    else:
        print(f"Matching columns for '{year_col}' not found.")

In [31]:
# Convert 'NA' strings to np.nan
for col in filtered_date_columns_1:
    df[col] = df[col].apply(convert_na)

original_cols = df.columns.tolist()

# Iterating over columns and creating date columns
for col in filtered_date_columns_1:
    if 'YYYY' in col:
        create_date_column_2(df, col, original_cols)

print(df.info())
df.head()

/tmp/ipykernel_29752/1088681976.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2018-05-10' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
/tmp/ipykernel_29752/1088681976.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-04-15' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
/tmp/ipykernel_29752/1088681976.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2014-10-22' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetim

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 990 entries, MaskID to DateRecordedDate3
dtypes: float64(421), int64(1), object(568)
memory usage: 1.4+ GB
None


,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,WhyAccessingParticipantRecord,ProtocolVersion,...,ContactDateDate,BirthDate,DateOfBirthDate,HbA1crecordDateDate,DateofAssessmentDate,DateCompletionDate,F2ClinicalDiagnosisDateDate,DateRecordedDate1,DateRecordedDate2,DateRecordedDate3
0,200082,03/30/2010,PRN,NaN,NaN,NaN,NaN,NaN,To enter or view data from a visit which occur...,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200082,06/13/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200185,07/06/2016,PRN,NaN,Initial Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200273,05/16/2012,PRN,NaN,Initial Screening Visit,Not applicable,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200273,06/14/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,3.0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
columns_to_drop = [col for col in filtered_date_columns_1]

len(columns_to_drop)

# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 972 entries, MaskID to DateRecordedDate3
dtypes: float64(409), int64(1), object(562)
memory usage: 1.4+ GB


## 1.4 day, mon, yea

In [33]:
contain_list_0 = ['mon'] 
# Example strings to exclude
not_contain_list_0 = ['month', 'mont', 'Monitor', 'Mono', 'PastSixMon', 'Whendidthepneumoniaoccur', 'ReasonsNotInterested', 'Monetary']     

# Converting each string in the lists to lower case
contain_list = [string.lower() for string in contain_list_0]
not_contain_list = [string.lower() for string in not_contain_list_0]

# Filtering columns with case-insensitive comparison
filtered_columns = [col for col in merged_df.columns if 
                    any(contain in col.lower() for contain in contain_list) and 
                    all(not_contain not in col.lower() for not_contain in not_contain_list)]

print("Found ", len(filtered_columns), "columns: ", filtered_columns)

filtered_df = merged_df[filtered_columns].dropna(how='all')

# Output the filtered DataFrame
print(filtered_df.info())

# Uncomment if you want to print top 20 values for each column
for col in filtered_columns:
    print(f"Top 20 values for column {col}:")
    print(merged_df[col].value_counts().nlargest(20))
    print()

Found  4 columns:  ['MechanisticSampleCollectionMon', 'ParticipantDateDispensedMon1_1', 'ParticipantDateDispensedMon2_1', 'ParticipantDateDispensedMon3_1']
<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 82819 to 194002
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   MechanisticSampleCollectionMon  1144 non-null   object
 1   ParticipantDateDispensedMon1_1  316 non-null    object
 2   ParticipantDateDispensedMon2_1  240 non-null    object
 3   ParticipantDateDispensedMon3_1  110 non-null    object
dtypes: object(4)
memory usage: 57.0+ KB
None
Top 20 values for column MechanisticSampleCollectionMon:
MechanisticSampleCollectionMon
Jun    114
Feb    106
Nov    105
Jul    104
Jan    103
Mar    102
Dec     98
May     97
Aug     93
Apr     93
Oct     69
Sep     60
Name: count, dtype: int64

Top 20 values for column ParticipantDateDispensedMon1_1:
ParticipantDateDispensedMon

In [34]:
col_list = ['ParticipantDateDispensedDay1_1', 'ParticipantDateDispensedMon1_1', 'ParticipantDateDispensedYea1_1',
'ParticipantDateDispensedDay2_1', 'ParticipantDateDispensedMon2_1', 'ParticipantDateDispensedYea2_1',
'ParticipantDateDispensedDay3_1', 'ParticipantDateDispensedMon3_1', 'ParticipantDateDispensedYea3_1',
'MechanisticSampleCollectionDay', 'MechanisticSampleCollectionMon', 'MechanisticSampleCollectionYea']

In [35]:
def create_date_column_3(df, year_col, original_cols):
    parts = year_col.split('Yea')
    base_col_name = parts[0]
    suffix = parts[1] if len(parts) > 1 else ''

    day_col = base_col_name + 'Day' + suffix
    month_col = base_col_name + 'Mon' + suffix

    if day_col in original_cols and month_col in original_cols:
        date_col = base_col_name + 'Date' + suffix
        problematic_rows = []
        for index, row in df.iterrows():
            try:
                if pd.isna(row[year_col]) or pd.isna(row[month_col]) or pd.isna(row[day_col]):
                    df.at[index, date_col] = np.nan
                else:
                    year = int(row[year_col])
                    month = int(month_to_num_1(row[month_col]))
                    day = int(row[day_col])
                    datetime_obj = pd.to_datetime(f"{year}-{month:02d}-{day:02d}")
                    df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
            except (ValueError, OverflowError) as e:
                problematic_rows.append((index, row[year_col], row[month_col], row[day_col]))
                df.at[index, date_col] = np.nan
        if problematic_rows:
            print("Problematic rows:")
            for row in problematic_rows:
                print(row)
    else:
        print(f"Matching columns for '{year_col}' not found.")

In [36]:
# Convert 'NA' strings to np.nan
for col in col_list:
    df[col] = df[col].apply(convert_na)

original_cols = df.columns.tolist()

# Iterating over columns and creating date columns
for col in col_list:
    if 'Yea' in col:
        create_date_column_3(df, col, original_cols)

print(df.info())
df.head()

/tmp/ipykernel_29752/2485481044.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-05-02' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
/tmp/ipykernel_29752/2485481044.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-05-02' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetime_obj.strftime('%Y-%m-%d')  # Format to only include the date
/tmp/ipykernel_29752/2485481044.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2016-08-12' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetim

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 976 entries, MaskID to MechanisticSampleCollectionDate
dtypes: float64(409), int64(1), object(566)
memory usage: 1.4+ GB
None


,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,WhyAccessingParticipantRecord,ProtocolVersion,...,DateofAssessmentDate,DateCompletionDate,F2ClinicalDiagnosisDateDate,DateRecordedDate1,DateRecordedDate2,DateRecordedDate3,ParticipantDateDispensedDate1_1,ParticipantDateDispensedDate2_1,ParticipantDateDispensedDate3_1,MechanisticSampleCollectionDate
0,200082,03/30/2010,PRN,NaN,NaN,NaN,NaN,NaN,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200082,06/13/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200185,07/06/2016,PRN,NaN,Initial Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200273,05/16/2012,PRN,NaN,Initial Screening Visit,Not applicable,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200273,06/14/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
len(col_list)

# Drop the columns from the DataFrame
df.drop(columns=col_list, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 964 entries, MaskID to MechanisticSampleCollectionDate
dtypes: float64(404), int64(1), object(559)
memory usage: 1.4+ GB


## 1.5 D, M, Y

In [38]:

def create_date_from_components(row):
    # Extracting year, month, and day from the row
    year = row['DateSignedInformedConsentFormY']
    month = row['DateSignedInformedConsentFormM']
    day = row['DateSignedInformedConsentFormD']

    # Check for NaN values and convert floats to integers
    if pd.isna(year) or pd.isna(month) or pd.isna(day):
        return pd.NaT
    try:
        year = int(year)
        day = int(day)
        # Constructing the date string
        date_str = f"{year}-{month}-{day:02d}"
        # Converting to datetime
        return pd.to_datetime(date_str, format='%Y-%b-%d').date()
    except (ValueError, TypeError):
        return pd.NaT

# Apply the function to each row to create the new date column
df['DateSignedInformedConsentFormDate'] = df.apply(create_date_from_components, axis=1)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 965 entries, MaskID to DateSignedInformedConsentFormDate
dtypes: float64(404), int64(1), object(560)
memory usage: 1.4+ GB


In [40]:
# Drop the columns from the DataFrame
df.drop(columns=['DateSignedInformedConsentFormM', 'DateSignedInformedConsentFormD', 'DateSignedInformedConsentFormY'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 962 entries, MaskID to DateSignedInformedConsentFormDate
dtypes: float64(402), int64(1), object(559)
memory usage: 1.4+ GB


## 1.6 Columns without indication of day, month, year

In [41]:
date_dict_1 ={ 'DateSignedNEWInformedConsentFoDate': ['DateSignedNEWInformedConsentFo', 'DateSignedNEWInformedConsentFo2', 'DateSignedNEWInformedConsentFo3'],
              'InterferonGammaReleaseAssayIGRDate': ['InterferonGammaReleaseAssayIGR', 'InterferonGammaReleaseAssayIGR3', 'InterferonGammaReleaseAssayIGR2']
             }

In [42]:
# Function to convert month from short name to number
def month_to_num_2(short_month):
    return {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }[short_month]

# Iterating over the dictionary to create new columns in filtered_df
for date_col, cols in date_dict_1.items():
    day_col, month_col, year_col = cols

    # Create a mask for rows where none of the day, month, or year values are missing
    mask = df[[day_col, month_col, year_col]].notna().all(axis=1)

    # Apply the mask to filter out rows with missing values
    valid_rows = df[mask]

    # Iterate over each row to handle individual errors
    for index, row in valid_rows.iterrows():
        try:
            # Combine the date components for the current row
            combined_date = str(int(row[day_col])) + '/' + \
                            str(month_to_num_2(row[month_col])) + '/' + \
                            str(int(row[year_col]))
            
            # Convert to datetime
            df.at[index, date_col] = pd.to_datetime(combined_date, format='%d/%m/%Y')
        except ValueError as e:
            # Print the problematic date and its index
            print(f"Error at index {index}: {combined_date}")
            print(f"Error message: {e}")

# Display the new DataFrame
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 964 entries, MaskID to InterferonGammaReleaseAssayIGRDate
dtypes: datetime64[ns](2), float64(402), int64(1), object(559)
memory usage: 1.4+ GB
None


,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,WhyAccessingParticipantRecord,ProtocolVersion,...,DateRecordedDate1,DateRecordedDate2,DateRecordedDate3,ParticipantDateDispensedDate1_1,ParticipantDateDispensedDate2_1,ParticipantDateDispensedDate3_1,MechanisticSampleCollectionDate,DateSignedInformedConsentFormDate,DateSignedNEWInformedConsentFoDate,InterferonGammaReleaseAssayIGRDate
0,200082,03/30/2010,PRN,NaN,NaN,NaN,NaN,NaN,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
1,200082,06/13/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
2,200185,07/06/2016,PRN,NaN,Initial Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
3,200273,05/16/2012,PRN,NaN,Initial Screening Visit,Not applicable,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
4,200273,06/14/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT


In [43]:
columns_to_drop = [column for sublist in date_dict_1.values() for column in sublist]
print(len(columns_to_drop))

# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)
df.info()

6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 958 entries, MaskID to InterferonGammaReleaseAssayIGRDate
dtypes: datetime64[ns](2), float64(398), int64(1), object(557)
memory usage: 1.4+ GB


## 1.7 time

In [44]:
time_dict = {'OGTTTestStartTime': ['OGTTTestStartTimeHrs', 'OGTTTestStartTimeMins'],
             'MMTTTestStartTime': ['MMTTTestStartTimeHrs', 'MMTTTestStartTimeMins'],
             'TimeOfCollection': ['TimeOfCollectionHH', 'TimeOfCollectionMM'],
             'TimeResultsProcessedByLab': ['TimeResultsProcessedByLabHH', 'TimeResultsProcessedByLabMM']
            }

In [45]:
def create_time_columns(df, time_dict):
    for new_col, time_cols in time_dict.items():
        hour_col, minute_col = time_cols

        if hour_col in df.columns and minute_col in df.columns:
            # Creating a time column with handling missing values and converting to integers
            df[new_col] = df.apply(lambda row: 
                                   f"{int(row[hour_col]):02d}:{int(row[minute_col]):02d}"
                                   if pd.notna(row[hour_col]) and pd.notna(row[minute_col]) and 
                                      row[hour_col] != '.' and row[minute_col] != '.'
                                   else np.nan, axis=1)
            # Converting string to datetime.time object
            df[new_col] = pd.to_datetime(df[new_col], format='%H:%M', errors='coerce').dt.time
        else:
            print(f"Columns for '{new_col}' not found. Hour column: {hour_col}, Minute column: {minute_col}")

    return df

df = create_time_columns(df, time_dict)
print(df.info())
df.head()

Columns for 'OGTTTestStartTime' not found. Hour column: OGTTTestStartTimeHrs, Minute column: OGTTTestStartTimeMins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 961 entries, MaskID to TimeResultsProcessedByLab
dtypes: datetime64[ns](2), float64(398), int64(1), object(560)
memory usage: 1.4+ GB
None


,MaskID,Visit_Dt,Visit,_22Jul2009VisitType,_15Aug2011VisitType,StorageOfResidualsDuringTrialN,_15Aug2011ConsentForResidualSa,WhichProtocolVersion,WhyAccessingParticipantRecord,ProtocolVersion,...,ParticipantDateDispensedDate1_1,ParticipantDateDispensedDate2_1,ParticipantDateDispensedDate3_1,MechanisticSampleCollectionDate,DateSignedInformedConsentFormDate,DateSignedNEWInformedConsentFoDate,InterferonGammaReleaseAssayIGRDate,MMTTTestStartTime,TimeOfCollection,TimeResultsProcessedByLab
0,200082,03/30/2010,PRN,NaN,NaN,NaN,NaN,NaN,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT
1,200082,06/13/2012,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT
2,200185,07/06/2016,PRN,NaN,Initial Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT
3,200273,05/16/2012,PRN,NaN,Initial Screening Visit,Not applicable,Yes,15August2011,To enter or view data from a visit which occur...,NaN,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT
4,200273,06/14/2018,PRN,NaN,Rescreen/Confirmatory Screening Visit,NaN,Yes,15August2011,To enter or view data from a visit which occur...,3.0,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT


In [ ]:
columns_to_drop = [column for sublist in time_dict.values() for column in sublist]
print(len(columns_to_drop))

# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)
df.info()

In [89]:
#df.to_csv('df_column_harmonization.csv', index = False)

In [347]:
df=pd.read_csv('df_column_harmonization.csv')

/tmp/ipykernel_29752/821926730.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,27,28,29,31,32,33,35,36,37,39,40,42,44,45,47,48,50,51,52,54,56,57,58,60,62,63,64,66,68,69,71,73,74,75,77,79,80,81,82,83,86,88,89,90,91,93,98,99,100,101,102,103,105,108,109,110,111,119,132,133,134,135,136,137,138,140,142,144,145,149,150,154,155,159,160,164,165,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,272,273,274,275,276,277,278,279,280,281,282,283,285,286,287,289,290,292,293,294,295,297,300,301,303,304,305,307,308,309,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,333,334,335,336,337,338,339,340,341,342,344,346,347,348,

In [348]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 961 entries, MaskID to TimeResultsProcessedByLab
dtypes: float64(480), int64(1), object(480)
memory usage: 1.4+ GB


# Missing values

In [349]:
df.dropna(axis=1, how='all', inplace=True)
df.dropna(axis=0, how='all', inplace=True)

In [350]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 898 entries, MaskID to TimeResultsProcessedByLab
dtypes: float64(417), int64(1), object(480)
memory usage: 1.3+ GB


In [351]:
#identify categorical columns and numeric columns first 

categorical_columns = df.select_dtypes(include=['object', 'category']).columns
numerical_columns = df.select_dtypes(include=['int', 'float']).columns
date_columns = df.select_dtypes(include=['datetime64']).columns

print("Categorical columns:", categorical_columns)
print('number of categorical columns:',len(categorical_columns))
print("Numerical columns:", numerical_columns) 
print('number of numerical columns:',len(numerical_columns))
print('number of date columns:',len(date_columns))

Categorical columns: Index(['Visit_Dt', 'Visit', '_22Jul2009VisitType', '_15Aug2011VisitType',
       'StorageOfResidualsDuringTrialN', '_15Aug2011ConsentForResidualSa',
       'WhichProtocolVersion', 'WhyAccessingParticipantRecord',
       'ProtocolVersion', 'WhichProtocolVersion2',
       ...
       'ParticipantDateDispensedDate1_1', 'ParticipantDateDispensedDate2_1',
       'ParticipantDateDispensedDate3_1', 'MechanisticSampleCollectionDate',
       'DateSignedInformedConsentFormDate',
       'DateSignedNEWInformedConsentFoDate',
       'InterferonGammaReleaseAssayIGRDate', 'MMTTTestStartTime',
       'TimeOfCollection', 'TimeResultsProcessedByLab'],
      dtype='object', length=480)
number of categorical columns: 480
Numerical columns: Index(['MaskID', 'severity', 'TimeOfCollectionHH', 'TimeOfCollectionMM',
       'DateResultsReportedByLabUnknow', 'TimeResultsProcessedByLabHH',
       'TimeResultsProcessedByLabMM', 'Rbcresults', 'HemoglobinResults',
       'HematocritResults',
    

In [352]:
#1st handle wrong datatype
#for any categorical variable, if it contains more than 20 unique values, then we can use common sense to judge 
# if it is better to convert it to 'int' datatype.
for i in categorical_columns:
    if len(df[i].unique())>20:
        print(i)
        print(df[i].unique())

Visit_Dt
['03/30/2010' '06/13/2012' '07/06/2016' ... '04/16/2017' '04/30/2017'
 '12/24/2017']
Visit
['PRN' '4 years' 'Baseline' '6 months' '7 years, 6 months'
 '2 years, 6 months' '3 years' '10 years' '1 year' '1 year, 6 months'
 '2 years' '9 years' '8 years' '8 years, 6 months' '4 years, 6 months'
 '5 years' '5 years, 6 months' '6 years, 6 months' '3 years, 6 months'
 '10 years, 6 months' '11 years' '11 years, 6 months' '7 years' '6 years'
 '12 years, 6 months' '9 years, 6 months' '12 years' '13 years'
 '13 years, 6 months' '14 years' '15 years' 'Screening' nan
 '14 years, 6 months' 'Initial Visit' '18 months' '30 months' '36 months'
 '42 months' '48 months' '12 months' '24 months' '54 months' '66 months'
 '60 months' '72 months' 'Day 1' '2 weeks' '4 weeks' '6 weeks' '8 weeks'
 '10 weeks' '12 weeks' '24 weeks' '36 weeks' '48 weeks' '72 weeks'
 '96 weeks' 'Rescreening #1' 'Day 2' '1 month' '2 months' '3 months'
 '7 months' '8 months' '9 months' 'Baseline 0A' 'Inital Visit'
 'Baseline 0

[nan 'S1533097' 'S1935758' 'S5615838' 'S1640133' 'S4846786' 'S0433752'
 'S6717775' 'S5841807' 'S1814629' 'S1554103' 'S2483999' 'S1053681'
 'S3200314' 'S0340369' 'S3557338' 'S5731760' 'S6543386' 'S0999863'
 'S0440976' 'S7838369' 'S9469446' 'S7791744' 'S0227527' 'S9807419'
 'S9624225' 'S5178731' 'S6821662' 'S9630354' 'S8126154' 'S5674198'
 'S2133963' 'S5866170' 'S3766049' 'S2809951' 'S4044305' 'S1254562'
 'S3016157' 'S9993464' 'S9566024' 'S2904988' 'S2913557' 'S7812868'
 'S3784904' 'S1115897' 'S3380093' 'S7162574' 'S4946490' 'S9248485'
 'S2497113' 'S4955693' 'S0336978' 'S8131949' 'S3425872' 'S9964424'
 'S1194463' 'S0881495' 'S6322552' 'S4818572' 'S9355288' 'S4213508'
 'S2153670' 'S7330533' 'S4150149' 'S6575909' 'S6614793' 'S5627409'
 'S4210845' 'S4152305' 'S7076635' 'S7759981' 'S4888011' 'S2192676'
 'S1354608' 'S2435562' 'S7238046' 'S3731262' 'S8689967' 'S8708655'
 'S3955252' 'S3757568' 'S1635699' 'S1351837' 'S2682545' 'S3896318'
 'S0188750' 'S7864944' 'S7542924' 'S3493022' 'S7433448' 'S

[nan '2015-11-30' '2011-10-13' '2018-08-16' '2010-01-11' '2018-11-16'
 '2013-12-30' '2019-06-26' '2019-01-07' '2015-11-05' '2017-04-14'
 '2016-08-01' '2015-06-09' '2014-10-06' '2008-09-09' '2016-02-01'
 '2019-02-08' '2014-11-04' '2017-04-08' '2018-04-24' '2011-07-05'
 '2012-07-20' '2019-09-21' '2011-01-24' '2017-04-04' '2019-11-25'
 '2020-01-16' '2013-07-18' '2012-02-10' '2017-03-21' '2017-05-31'
 '2011-10-15' '2016-11-25' '2018-07-27' '2017-12-22' '2018-05-09'
 '2020-05-12' '2018-11-21' '2018-08-30' '2010-01-26' '2009-05-03'
 '2009-11-02' '2010-07-27' '2013-01-26' '2013-08-23' '2014-08-25'
 '2018-02-06' '2012-08-08' '2018-08-07' '2016-03-24' '2016-10-19'
 '2017-12-27' '2018-01-27' '2018-12-27' '2015-07-16' '2013-03-05'
 '2011-07-28' '2015-04-26' '2018-11-28' '2017-03-29' '2019-04-04'
 '2018-09-14' '2019-09-26' '2009-06-02' '2010-01-25' '2018-03-27'
 '2017-12-01' '2018-06-29' '2011-05-05' '2014-12-17' '2018-09-12'
 '2013-07-22' '2013-05-16' '2016-08-09' '2018-04-30' '2019-03-11'
 '2016

In [353]:
for i in numerical_columns:
    if len(df[i].unique())<10:
        print(i)
        print(df[i].unique())

severity
[nan  3.  2.  1.  5.  4.]
DateResultsReportedByLabUnknow
[nan  1.]
ReasonsNotInterested_Conflicting
[nan  1.]
ReasonsNotInterested_Other
[nan  1.]
ParticipantWasContac_AffiliateS1
[nan  1.]
DateParticipantwascontacted1_3
[  nan 2019. 2016. 2018. 2017. 2020.]
ParticipantWasCo_ClinicalCen1_1
[nan  1.]
MethodofContactA_Telephone1_1
[nan  1.]
MethodofContactA_Email1_1
[nan  1.]
DegreeOfRelative_1st
[nan  1.  0.]
DegreeOfRelative_2ndor3rd
[nan  0.  1.]
Race_AmericanIndianorAlaskaNat
[nan  0.  1.]
Race_Asian
[nan  0.  1.]
Race_BlackorAfricanAmerican
[nan  0.  1.]
Race_NativeHawaiianorOtherPaci
[nan  0.  1.]
Race_Refused
[nan  0.  1.]
Race_Unknownornotreported
[nan  0.  1.]
Race_White
[nan  1.  0.]
FirstHearAboutTrialN_ExistingTri
[nan  0.  1.]
FirstHearAboutTrialN_FamilyFrien
[nan  0.  1.]
FirstHearAboutTrialN_Brochure
[nan  0.  1.]
FirstHearAboutTrialN_Healthcarep
[nan  0.  1.]
FirstHearAboutTrialN_Tollfreepho
[nan  0.  1.]
FirstHearAboutTrialN_NewspaperMa
[nan  0.  1.]
FirstHearAb

In [354]:
#these columns only have  value of 0 in it， so we can just drop them
element_to_remove =['EBVVCAIGM','EBVVCAIGG','EBNA','CMVIGM','HEPC','CMVIGG','HIV', 'HEPBC','HEPB','DR3','DR4','HLA','HLAa','HLAb',
                   'AEAssociations_Developmentofacon','AEAssociations_Developmentofaper','AEAssociations_Death',
                    'AEAssociations_Isanotherconditio','AEAssociations_OtherSeriousImpor']

for i in element_to_remove:
    df=df.drop(i,axis=1)


In [355]:
#columns should be coverted to categorical variables 
to_convert_cat=['DateResultsReportedByLabUnknow',
            'DegreeOfRelative_1st',
            'DegreeOfRelative_2ndor3rd',
            'Race_AmericanIndianorAlaskaNat',
            'Race_Asian',
            'Race_BlackorAfricanAmerican',
            'Race_NativeHawaiianorOtherPaci',
            'Race_Refused',
            'Race_Unknownornotreported',
            'Race_White',
            'FirstHearAboutTrialN_ExistingTri',
            'FirstHearAboutTrialN_FamilyFrien',
            'FirstHearAboutTrialN_Brochure',
            'FirstHearAboutTrialN_Healthcarep',
            'FirstHearAboutTrialN_Tollfreepho',
            'FirstHearAboutTrialN_NewspaperMa',
            'FirstHearAboutTrialN_Poster',
            'FirstHearAboutTrialN_Camp',
            'FirstHearAboutTrialN_MeetingPres',
            'FirstHearAboutTrialN_RadioTV',
            'FirstHearAboutTrialN_Internet',
            'FirstHearAboutTrialN_Other',
            'IsHeightMeasured','IsWeightMeasured',
            'Rbcnotdone',
            'HemoglobinNotDone','HematocritNotDone','MCVNotDone',
            'WBCNotDone','bloodpressureDiastolicNotDone','WeightNotDone',
            'HeightNotDone']


In [356]:
#convert some numerical variables to categorical variables 
for i in to_convert_cat:
    df[i] = df[i].astype(object)


In [357]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
numerical_columns = df.select_dtypes(include=['int', 'float']).columns
date_columns = df.select_dtypes(include=['datetime64[ns]']).columns
#statistic the columns with <50% missing value; >=50% & <=99% missing value; >99% missing value
missing_60=[]
drop_95=[]
between_50_99=[]
for i in numerical_columns:
    count_null=df[i].isnull().sum()
    print(i)
    all1=len(df[i])
    null_per=count_null/all1
    print(null_per*100,'%')
    if null_per*100<50:
        missing_60.append(i)
    if null_per*100>=50 and null_per*100<=99.5:
        between_50_99.append(i)
    if null_per*100>99.5:
        drop_95.append(i)
     #   df = df.drop(i, axis=1)


MaskID
0.0 %
severity
99.54549637521856 %
TimeOfCollectionHH
99.82668534443093 %
TimeOfCollectionMM
99.82668534443093 %
TimeResultsProcessedByLabHH
99.86502929477807 %
TimeResultsProcessedByLabMM
99.86502929477807 %
Rbcresults
98.78781991635907 %
HemoglobinResults
98.78833116903036 %
HematocritResults
98.78884242170164 %
MCVresults
98.78628615834518 %
PlateletCountResultsNormal
98.79088743238682 %
MCHresults
98.78884242170164 %
MCHCresults
98.78935367437295 %
WBCresults
98.78935367437295 %
NeutrophilsResults
99.30623012505241 %
NeutrophilsResultsOR
99.38803055245964 %
LymphocytesResults
99.28782502888578 %
LymphocytesResultsOR
99.37576048834855 %
MonocytesResults
99.28987003957096 %
MonocytesResultsOR
99.37627174101985 %
EosinophilsResults
99.28987003957096 %
EosinophilsResultsOR
99.37473798300596 %
BasophilsResults
99.29089254491355 %
BasophilsResultsOR
99.37627174101985 %
ReasonsNotInterested_Conflicting
99.9355821634168 %
ReasonsNotInterested_Other
99.94836348019919 %
ParticipantWas

In [358]:
df=df.drop(columns=drop_95)

In [359]:
len(missing_60)

67

In [360]:
len(drop_95)

258

In [361]:
len(between_50_99)

42

In [362]:
df.shape

(195598, 621)

In [363]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns


In [364]:
# get date columns out of the categorical columns. Use the remaining categorical columns for encoding
date_col = [col for col in categorical_columns]

def separate_columns_by_date(column_names):
    
    date_columns = []
    non_date_columns = []

    # Regular expression pattern for common date-related keywords
    date_pattern = re.compile(r'date|time', re.IGNORECASE)

    for column_name in column_names:
        if date_pattern.search(column_name):
            date_columns.append(column_name)
        else:
            non_date_columns.append(column_name)

    return date_columns, non_date_columns


date_columns, non_date_columns = separate_columns_by_date(date_col)
#these two columns are also date columns
date_columns.extend(['Visit_Dt'])

element_to_remove =['Visit_Dt']
for i in element_to_remove:
    if i in non_date_columns:
        non_date_columns.remove(i)


print("Columns that likely represent dates:")
print(date_columns)

print("\nColumns that do not appear to represent dates:")
print(non_date_columns)


Columns that likely represent dates:
['DateResultsReportedByLabUnknow', 'willingToConsiderAtALaterTime', 'DateParticipantwascontacted1_2', 'HospitalizedAtDiagnosisTime', 'Date_of_Registration', 'Date_of_Study_Start', 'RememberDateOfMostRecentHbA1c', 'Date_of_Diagnosis', 'aedate', 'AEFollowUpDate', 'DateOfRegistration', 'DiabeticKetoacidosisTimeOfDiag', 'MethylprednisoneGivenDoseTimeM', 'Date_of_Draw', 'death_date', 'ReviewDate', 'Date_Run', 'Datewritteninformedconsentasse2', '_22Jul2009ConsentSignedDate', '_15Aug2011ConsentDateDate', 'Fall2017ConsentDateDate', 'Spring2019ConsentDateDate', 'DateOfBloodDrawDate', 'DateReportedDate', 'ScreeningDate', 'ConsentSignedDate', 'DateConsentSignedDate', 'SamplesCollectionDate', 'DateStatusChangeDate', 'DoRDate', 'DoWDate', 'DiagnosisDate', 'InsulinTreatmentStartedDate', 'AdmissionDate', 'DischargeDate', 'GlucoseDate1_1', 'GlucoseDate2_1', 'GlucoseDate3_1', 'HbA1cDate', 'ProtcolDeviationDate', 'AssessDateDate', 'StartDateDate', 'StopDateDate', 'La

In [365]:
len(date_columns)+len(non_date_columns)

512

In [366]:
numerical_columns = df.select_dtypes(include=['int', 'float']).columns


In [367]:
len(numerical_columns)

109

In [368]:
#fill in zero for the null values in Date/Time columns
df[date_columns]=df[date_columns].fillna(0)

In [369]:
#encode all the cateogrical variables except date and time

embed=[]
for i in non_date_columns:
    print(i)
    df[i]=df[i].astype('str')
    column_name=i
    value_counts = df[column_name].value_counts()
    #print(value_counts)
    my_dict={}
    key=df[column_name].unique()
    print('unique values:',key)
    null_count = df[column_name].isnull().sum()
    #print(f"Number of null values in column '{column_name}': {null_count}")
    le = LabelEncoder()
    df[column_name] = le.fit_transform(df[column_name])
    value=df[column_name].unique()
    # Fill the dictionary with keys and values using a loop
    for k, v in zip(key, value):
        my_dict[k] = v

    dict1={'Column':column_name,'Details':my_dict}
    value_counts = df[column_name].value_counts()
   # print(value_counts)
    null_count = df[column_name].isnull().sum()
    print(f"Number of null values after preprocessing in column '{column_name}': {null_count}")
    
    embed.append(dict1)

Visit
unique values: ['PRN' '4 years' 'Baseline' '6 months' '7 years, 6 months'
 '2 years, 6 months' '3 years' '10 years' '1 year' '1 year, 6 months'
 '2 years' '9 years' '8 years' '8 years, 6 months' '4 years, 6 months'
 '5 years' '5 years, 6 months' '6 years, 6 months' '3 years, 6 months'
 '10 years, 6 months' '11 years' '11 years, 6 months' '7 years' '6 years'
 '12 years, 6 months' '9 years, 6 months' '12 years' '13 years'
 '13 years, 6 months' '14 years' '15 years' 'Screening' 'nan'
 '14 years, 6 months' 'Initial Visit' '18 months' '30 months' '36 months'
 '42 months' '48 months' '12 months' '24 months' '54 months' '66 months'
 '60 months' '72 months' 'Day 1' '2 weeks' '4 weeks' '6 weeks' '8 weeks'
 '10 weeks' '12 weeks' '24 weeks' '36 weeks' '48 weeks' '72 weeks'
 '96 weeks' 'Rescreening #1' 'Day 2' '1 month' '2 months' '3 months'
 '7 months' '8 months' '9 months' 'Baseline 0A' 'Inital Visit'
 'Baseline 0B' '14 weeks' '16 weeks' '18 weeks' '20 weeks' '22 weeks']
Number of null val

Number of null values after preprocessing in column 'TN01ParticipantStatusSectionB': 0
Rbcresults_Units
unique values: ['nan' 'x10^12 cells/L']
Number of null values after preprocessing in column 'Rbcresults_Units': 0
RBCResultsNormal
unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'RBCResultsNormal': 0
RBCResultAbormalClinicallySign
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'RBCResultAbormalClinicallySign': 0
HemoglobinResults_Units
unique values: ['nan' 'mg/dL']
Number of null values after preprocessing in column 'HemoglobinResults_Units': 0
HemoglobinResultsNormal
unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'HemoglobinResultsNormal': 0
HemoglobinResultAbormalClinica
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'HemoglobinResultAbormalClinica': 0
HematocritResults_Units
unique values: ['nan' '% of blood']
Number of null v

Number of null values after preprocessing in column 'ParticipantLocation': 0
Sex
unique values: ['nan' 'Male' 'Female']
Number of null values after preprocessing in column 'Sex': 0
Ethnicity
unique values: ['nan' 'Not Hispanic or Latino' 'Unknown' 'Hispanic or Latino'
 'Not Hispanic, Latino, or Spanish Origin'
 'Hispanic, Latino, or Spanish Origin']
Number of null values after preprocessing in column 'Ethnicity': 0
Race_AmericanIndianorAlaskaNat
unique values: ['nan' '0.0' '1.0']
Number of null values after preprocessing in column 'Race_AmericanIndianorAlaskaNat': 0
Race_Asian
unique values: ['nan' '0.0' '1.0']
Number of null values after preprocessing in column 'Race_Asian': 0
Race_BlackorAfricanAmerican
unique values: ['nan' '0.0' '1.0']
Number of null values after preprocessing in column 'Race_BlackorAfricanAmerican': 0
Race_NativeHawaiianorOtherPaci
unique values: ['nan' '0.0' '1.0']
Number of null values after preprocessing in column 'Race_NativeHawaiianorOtherPaci': 0
Race_Refuse

Number of null values after preprocessing in column 'BloodSamplesFutureTestingPermi': 0
BloodTestForFindOut
unique values: ['nan'
 'I have autoantibodies which means I am at risk of getting Type 1 diabetes'
 'I have Type 1 diabetes' 'I don?t know' 'I have Type 2 diabetes']
Number of null values after preprocessing in column 'BloodTestForFindOut': 0
CommunicationWhenNegAutoantibo
unique values: ['nan' 'Letter' 'Phone Call'
 'We will only notify you if you are positive for autoantibodies'
 'I don?t know']
Number of null values after preprocessing in column 'CommunicationWhenNegAutoantibo': 0
InformedTestResultWithin
unique values: ['nan' '1 week' '4-6 weeks' '3 months' '1 year']
Number of null values after preprocessing in column 'InformedTestResultWithin': 0
PermissionToControlIfTheyTestA
unique values: ['nan' 'Yes' 'No' 'Unknown']
Number of null values after preprocessing in column 'PermissionToControlIfTheyTestA': 0
ScreeningSiteName
unique values: ['nan' 'Indiana University-Riley Hos

Number of null values after preprocessing in column 'ScreeningSiteName': 0
shipspecimencollectionkit
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'shipspecimencollectionkit': 0
RelativeWithT1D1_1
unique values: ['nan' 'FS=Brother/Sister' 'CH=Child' 'C=Cousin' 'P=Parent'
 'GP=Grandparent' 'AU=Aunt/Uncle' 'NT=Non-identical Twin'
 'HS=Half Brother/Sister' 'IT=Identical Twin' 'N=Niece/Nephew'
 'HC=Half-Cousin']
Number of null values after preprocessing in column 'RelativeWithT1D1_1': 0
SexOfRelativeWithT1D1_1
unique values: ['nan' 'Female' 'Male']
Number of null values after preprocessing in column 'SexOfRelativeWithT1D1_1': 0
RelativeWithT1D2_1
unique values: ['nan' 'FS=Brother/Sister' 'AU=Aunt/Uncle' 'P=Parent' 'GP=Grandparent'
 'C=Cousin' 'CH=Child' 'HS=Half Brother/Sister' 'N=Niece/Nephew'
 'NT=Non-identical Twin' 'HC=Half-Cousin' 'IT=Identical Twin']
Number of null values after preprocessing in column 'RelativeWithT1D2_1': 0
SexOfRelativeWithT1

...

unique values: ['nan' 'Not Applicable'
 '                                                                                                   .'
 'Yes' 'No']
Number of null values after preprocessing in column 'Eventreappearafterreintroduct': 0
AEReasonForFollowup
unique values: ['nan' 'New information, progression of event'
 'Correction of initial report'
 'Additional information requested by the medical monitor']
Number of null values after preprocessing in column 'AEReasonForFollowup': 0
DiagnosedAutoImmuneDisease
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'DiagnosedAutoImmuneDisease': 0
Asthma
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'Asthma': 0
Allergies
unique values: ['nan' 'No' 'Yes' 'Unknown']
Number of null values after preprocessing in column 'Allergies': 0
Eczema
unique values: ['nan' 'No' 'Yes' 'Unknown']
Number of null values after preprocessing in column 'Eczema': 0
OtherInfections
uniq

unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'EligibileToBeScreenedForTheStu': 0
SexGenderPerson
unique values: ['nan' 'Female' 'Male']
Number of null values after preprocessing in column 'SexGenderPerson': 0
HospitalizedOtherthanDiabetes
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'HospitalizedOtherthanDiabetes': 0
AdmittedHospitalDiagnosis
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'AdmittedHospitalDiagnosis': 0
IGRATestPositive
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'IGRATestPositive': 0
PregTestPos
unique values: ['nan' 'No' 'Not applicable']
Number of null values after preprocessing in column 'PregTestPos': 0
FebrileIllness10Days
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'FebrileIllness10Days': 0
AnyGrade3OrGreaterProblems
unique values: ['nan' 'No']
Number of null va

unique values: ['nan' '1.0']
Number of null values after preprocessing in column 'Rbcnotdone': 0
HemoglobinNotDone
unique values: ['nan' '1.0']
Number of null values after preprocessing in column 'HemoglobinNotDone': 0
HematocritNotDone
unique values: ['nan' '1.0']
Number of null values after preprocessing in column 'HematocritNotDone': 0
MCVNotDone
unique values: ['nan' '1.0']
Number of null values after preprocessing in column 'MCVNotDone': 0
WBCNotDone
unique values: ['nan' '1.0']
Number of null values after preprocessing in column 'WBCNotDone': 0
HBsAb
unique values: ['nan' '.82124453493538' '.709174699088733' '.736969197661222'
 '.731585569038634' '7.4304799350730602E-2' '7.435682744849979'
 '9.02081482117741' '9.30225347381084' '8.44279861677383'
 '4.8077813311177' '.782815764827669' '<LLOD' '3.17938125959935'
 '.19159722928677' '.7347872248337569' '.721980761637899'
 '1.23491059371597' '1.42778905783334' '1.22269255483472'
 '3.65322652221296' '3.7651294650481' '2.29408724068117'

Number of null values after preprocessing in column 'HeightNotDone': 0
Wasaphysicalexamperformedatthi
unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'Wasaphysicalexamperformedatthi': 0
HEENT
unique values: ['nan' 'Normal' 'Abnormal' 'Not Assessed']
Number of null values after preprocessing in column 'HEENT': 0
NeckThyroid
unique values: ['nan' 'Normal' 'Abnormal' 'Not Assessed']
Number of null values after preprocessing in column 'NeckThyroid': 0
Heart
unique values: ['nan' 'Normal' 'Not Assessed' 'Abnormal']
Number of null values after preprocessing in column 'Heart': 0
Lungs
unique values: ['nan' 'Normal' 'Abnormal' 'Not Assessed']
Number of null values after preprocessing in column 'Lungs': 0
Pulses
unique values: ['nan' 'Normal' 'Abnormal' 'Not Assessed']
Number of null values after preprocessing in column 'Pulses': 0
Musculoskeletal
unique values: ['nan' 'Normal' 'Abnormal' 'Not Assessed']
Number of null values after preprocessing in column '

Number of null values after preprocessing in column 'SignsOfHypoglycemia': 0
OtherProblems
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'OtherProblems': 0
StudyDrugReturned
unique values: ['nan' 'No' 'Yes']
Number of null values after preprocessing in column 'StudyDrugReturned': 0
StudyDrugDispensed
unique values: ['nan' 'Yes' 'No']
Number of null values after preprocessing in column 'StudyDrugDispensed': 0
ParticipantReceiveStudyDrug
unique values: ['nan' 'At Clinical Center' 'By Courier']
Number of null values after preprocessing in column 'ParticipantReceiveStudyDrug': 0
MedicationPackNumber1_1
unique values: ['nan' 'BC#3' 'BC#5' '7' '02' 'BC04' '03' '05' '01' 'BC#4' 'BC#6' 'BC 01'
 'BC03' '04' 'BC#02' '06' '07' '3' 'Oral Insulin Capsule 250mg BC 05'
 'Oral Insulin capsule 250mg Bottle no. 2' '4' 'BC-03' 'BC-04' 'BC-05'
 'BC-01' 'BC-02' 'BC-06' '1' 'BC#03' '16B0082'
 'Oral Insulin Capsules 250g strength' '05 Insulin capsules 250mg' 'BC#2'
 'B

In [370]:
embed

[{'Column': 'Visit',
  'Details': {'PRN': 70,
   '4 years': 36,
   'Baseline': 63,
   '6 months': 44,
   '7 years, 6 months': 52,
   '2 years, 6 months': 24,
   '3 years': 30,
   '10 years': 4,
   '1 year': 1,
   '1 year, 6 months': 2,
   '2 years': 23,
   '9 years': 60,
   '8 years': 57,
   '8 years, 6 months': 58,
   '4 years, 6 months': 37,
   '5 years': 41,
   '5 years, 6 months': 42,
   '6 years, 6 months': 47,
   '3 years, 6 months': 31,
   '10 years, 6 months': 5,
   '11 years': 6,
   '11 years, 6 months': 7,
   '7 years': 51,
   '6 years': 46,
   '12 years, 6 months': 11,
   '9 years, 6 months': 61,
   '12 years': 10,
   '13 years': 12,
   '13 years, 6 months': 13,
   '14 years': 15,
   '15 years': 17,
   'Screening': 72,
   'nan': 73,
   '14 years, 6 months': 16,
   'Initial Visit': 69,
   '18 months': 19,
   '30 months': 32,
   '36 months': 33,
   '42 months': 38,
   '48 months': 39,
   '12 months': 8,
   '24 months': 27,
   '54 months': 43,
   '66 months': 49,
   '60 months'

In [190]:
#save it to create data dictionary
with open('encoding.pickle', 'wb') as file:
    pickle.dump(embed, file)

In [371]:
len(missing_60)

67

In [372]:
len(drop_95)

258

In [373]:
len(between_50_99)

42

In [374]:
# Loop through the specified columns and add a missing indicator column for each
# 1 means misssing , 0 means not missing
for col in between_50_99:
    df[f'{col}_missing_indicator'] = df[col].isnull().astype(int)

/tmp/ipykernel_29752/420081964.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_missing_indicator'] = df[col].isnull().astype(int)
/tmp/ipykernel_29752/420081964.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_missing_indicator'] = df[col].isnull().astype(int)
/tmp/ipykernel_29752/420081964.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1)

In [375]:
len(missing_60)

67

In [376]:
df.shape

(195598, 663)

In [379]:
missing_60

['MaskID',
 'GAD65H_y',
 'MIAA',
 'IA-2H',
 'ZNT8',
 'ICA512',
 'ICA',
 'GAD65',
 'GLU60',
 'PEP120',
 'PEP60',
 'INST-10',
 'PEP-10',
 'INST60',
 'INST0',
 'GLU30',
 'HbA1c',
 'GLU-10',
 'GLU0',
 'PEP30',
 'INST30',
 'INST120',
 'GLU90',
 'PEP90',
 'GLU120',
 'INST90',
 'PEP0',
 'INS0',
 'INS120',
 'INS-10',
 'INS60',
 'INS30',
 'INS90',
 'GLU15',
 'PEP15',
 'PEP240',
 'GLU240',
 'PEP150',
 'GLU180',
 'PEP180',
 'GLU150',
 'PEP210',
 'EBVVL',
 'CMVVL',
 'GLU210',
 'CD8P',
 'CD8A',
 'CD4P',
 'CD4CD8',
 'CD4A',
 'TPRO',
 'TBIL',
 'SOD',
 'POT',
 'GLU',
 'GLOIgG',
 'DBIL',
 'CREA',
 'CHL',
 'BIC',
 'ALT',
 'ALP',
 'ALB',
 'UREA',
 'AST',
 'CPEP',
 'INST']

In [384]:
# Initialize the KNN imputer
imputer = KNNImputer(n_neighbors=5)  # You can adjust 'n_neighbors' as needed
# Impute missing values using the KNN imputer for each numeric column with less than 50% missing value
j=0
for column in missing_60[1:]:
    print(column)
    print(j)
    j+=1
    df[column] = imputer.fit_transform(df[column].values.reshape(-1, 1))

GAD65H_y
0
MIAA
1
IA-2H
2
ZNT8
3
ICA512
4
ICA
5
GAD65
6
GLU60
7
PEP120
8
PEP60
9
INST-10
10
PEP-10
11
INST60
12
INST0
13
GLU30
14
HbA1c
15
GLU-10
16
GLU0
17
PEP30
18
INST30
19
INST120
20
GLU90
21
PEP90
22
GLU120
23
INST90
24
PEP0
25
INS0
26
INS120
27
INS-10
28
INS60
29
INS30
30
INS90
31
GLU15
32
PEP15
33
PEP240
34
GLU240
35
PEP150
36
GLU180
37
PEP180
38
GLU150
39
PEP210
40
EBVVL
41
CMVVL
42
GLU210
43
CD8P
44
CD8A
45
CD4P
46
CD4CD8
47
CD4A
48
TPRO
49
TBIL
50
SOD
51
POT
52
GLU
53
GLOIgG
54
DBIL
55


KeyboardInterrupt: 

In [383]:
df.to_csv('df_knn1.csv',index=False)

In [3]:
df=pd.read_csv('df_knn1.csv')

/tmp/ipykernel_10253/1242637108.py:1: DtypeWarning: Columns (1,79,84,263,296,298,321,332,335,343,387,398,423,439,443,444,450,452,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('df_knn1.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 663 entries, MaskID to RespiratoryRateBPM_missing_indicator
dtypes: float64(109), int64(482), object(72)
memory usage: 989.4+ MB


In [6]:
len(df['MaskID'].unique())

37683

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195598 entries, 0 to 195597
Columns: 939 entries, MaskID to RespiratoryRateBPM_missing_indicator
dtypes: datetime64[ns](2), float64(335), int64(42), object(560)
memory usage: 1.4+ GB


In [ ]:
df.to_csv('AI_Ready.csv',index=False)

In [203]:
df.shape

(195598, 663)

In [238]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns


In [239]:
len(categorical_columns)

511

In [228]:
# get date columns out of the categorical columns. Use the remaining categorical columns for encoding
date_col = [col for col in categorical_columns]

def separate_columns_by_date(column_names):
    
    date_columns = []
    non_date_columns = []

    # Regular expression pattern for common date-related keywords
    date_pattern = re.compile(r'date|time', re.IGNORECASE)

    for column_name in column_names:
        if date_pattern.search(column_name):
            date_columns.append(column_name)
        else:
            non_date_columns.append(column_name)

    return date_columns, non_date_columns


date_columns, non_date_columns = separate_columns_by_date(date_col)
#these two columns are also date columns
date_columns.extend(['Visit_Dt'])

element_to_remove =['Visit_Dt']
for i in element_to_remove:
    if i in non_date_columns:
        non_date_columns.remove(i)


print("Columns that likely represent dates:")
print(date_columns)

print("\nColumns that do not appear to represent dates:")
print(non_date_columns)

Columns that likely represent dates:
['willingToConsiderAtALaterTime', 'DateParticipantwascontacted1_2', 'HospitalizedAtDiagnosisTime', 'Date_of_Registration', 'Date_of_Study_Start', 'RememberDateOfMostRecentHbA1c', 'Date_of_Diagnosis', 'aedate', 'AEFollowUpDate', 'DateOfRegistration', 'DiabeticKetoacidosisTimeOfDiag', 'MethylprednisoneGivenDoseTimeM', 'Date_of_Draw', 'death_date', 'ReviewDate', 'Date_Run', 'Datewritteninformedconsentasse2', '_22Jul2009ConsentSignedDate', '_15Aug2011ConsentDateDate', 'Fall2017ConsentDateDate', 'Spring2019ConsentDateDate', 'DateOfBloodDrawDate', 'DateReportedDate', 'ScreeningDate', 'ConsentSignedDate', 'DateConsentSignedDate', 'SamplesCollectionDate', 'DateStatusChangeDate', 'DoRDate', 'DoWDate', 'DiagnosisDate', 'InsulinTreatmentStartedDate', 'AdmissionDate', 'DischargeDate', 'GlucoseDate1_1', 'GlucoseDate2_1', 'GlucoseDate3_1', 'HbA1cDate', 'ProtcolDeviationDate', 'AssessDateDate', 'StartDateDate', 'StopDateDate', 'LastHbA1cDate', 'InformedConsentDate

In [237]:
df.shape

(195598, 621)

In [240]:
621-511

110

In [229]:
len(non_date_columns)

439

In [230]:
len(date_columns)

72

In [231]:
data=embed
df_list = []

# Loop through the list of dictionaries and add each 'Column' and 'Details' to the DataFrame
for item in data:
    #print(item)
    column_name = item['Column']
    details_dict = item['Details']
    
    # Convert the details_dict to a string representation
    details_str = str(details_dict)
    
    # Create a dictionary with 'Column' and 'Details' keys
    row_dict = {'Variable Name': column_name, 'Variable Type':'object','Measurement Units as applicable':None,'Corresponding Code List': details_str,'Missingness Representation':'KNN Imputation'}
    
    # Append the dictionary to the list
    df_list.append(row_dict)

df_encode = pd.DataFrame(df_list)



In [232]:
df_encode

,Variable Name,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation
0,Visit,object,None,"{'PRN': 70, '4 years': 36, 'Baseline': 63, '6 ...",KNN Imputation
1,_22Jul2009VisitType,object,None,"{'nan': 4, 'Initial Screening Visit': 1, 'Resc...",KNN Imputation
2,_15Aug2011VisitType,object,None,"{'nan': 5, 'Rescreen/Confirmatory Screening Vi...",KNN Imputation
3,StorageOfResidualsDuringTrialN,object,None,"{'nan': 1, 'Not applicable': 0, 'no': 2, 'yes'...",KNN Imputation
4,_15Aug2011ConsentForResidualSa,object,None,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",KNN Imputation
...,...,...,...,...,...
434,Havetherebeenanychangesinconco,object,None,"{'nan': 2, 'No': 0, 'Yes': 1}",KNN Imputation
435,Istheparticipantcurrentlytakin,object,None,"{'nan': 2, 'Yes': 1, 'No': 0}",KNN Imputation
436,HbA1c_Units,object,None,"{'nan': 1, 'Hb%': 0}",KNN Imputation
437,Height_cm_units,object,None,{'cm': 0},KNN Imputation


In [233]:
len(date_columns)

72

In [234]:
df_encode1 = pd.DataFrame(date_columns, columns=['Variable Name'])
df_encode1['Variable Type']='object'
df_encode1['Measurement Units as applicable']=None
df_encode1['Corresponding Code List']=None
df_encode1['Missingness Representation']=0
df_encode1

,Variable Name,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation
0,willingToConsiderAtALaterTime,object,None,None,0
1,DateParticipantwascontacted1_2,object,None,None,0
2,HospitalizedAtDiagnosisTime,object,None,None,0
3,Date_of_Registration,object,None,None,0
4,Date_of_Study_Start,object,None,None,0
...,...,...,...,...,...
67,InterferonGammaReleaseAssayIGRDate,object,None,None,0
68,MMTTTestStartTime,object,None,None,0
69,TimeOfCollection,object,None,None,0
70,TimeResultsProcessedByLab,object,None,None,0


In [235]:
df_encode2 = pd.DataFrame(['DateSignedNEWInformedConsentFoDate','InterferonGammaReleaseAssayIGRDate'], columns=['Variable Name'])
df_encode2['Variable Type']='datetime64'
df_encode2['Measurement Units as applicable']='NA'
df_encode2['Corresponding Code List']='NA'
df_encode2['Missingness Representation']=0
df_encode2

,Variable Name,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation
0,DateSignedNEWInformedConsentFoDate,datetime64,NA,NA,0
1,InterferonGammaReleaseAssayIGRDate,datetime64,NA,NA,0


In [241]:
len(missing_60)

67

In [242]:
df_encode3 = pd.DataFrame(missing_60, columns=['Variable Name'])
df_encode3['Variable Type']='float64'
df_encode3['Measurement Units as applicable']=None
df_encode3['Corresponding Code List']=None
df_encode3['Missingness Representation']='KNN Imputation'
df_encode3

,Variable Name,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation
0,MaskID,float64,None,None,KNN Imputation
1,GAD65H_y,float64,None,None,KNN Imputation
2,MIAA,float64,None,None,KNN Imputation
3,IA-2H,float64,None,None,KNN Imputation
4,ZNT8,float64,None,None,KNN Imputation
...,...,...,...,...,...
62,ALB,float64,None,None,KNN Imputation
63,UREA,float64,None,None,KNN Imputation
64,AST,float64,None,None,KNN Imputation
65,CPEP,float64,None,None,KNN Imputation


In [244]:
42+67

109

In [243]:
len(between_50_99)

42

In [245]:
df_encode4 = pd.DataFrame(between_50_99, columns=['Variable Name'])
df_encode4['Variable Type']='float64'
df_encode4['Measurement Units as applicable']=None
df_encode4['Corresponding Code List']=None
df_encode4['Missingness Representation']='Use missing indicator'
df_encode4

,Variable Name,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation
0,Rbcresults,float64,None,None,Use missing indicator
1,HemoglobinResults,float64,None,None,Use missing indicator
2,HematocritResults,float64,None,None,Use missing indicator
3,MCVresults,float64,None,None,Use missing indicator
4,PlateletCountResultsNormal,float64,None,None,Use missing indicator
5,MCHresults,float64,None,None,Use missing indicator
6,MCHCresults,float64,None,None,Use missing indicator
7,WBCresults,float64,None,None,Use missing indicator
8,NeutrophilsResults,float64,None,None,Use missing indicator
9,NeutrophilsResultsOR,float64,None,None,Use missing indicator


In [346]:
df_encode.to_csv('df_encode.csv',index=False)
df_encode1.to_csv('df_encode1.csv',index=False)
df_encode2.to_csv('df_encode2.csv',index=False)
df_encode3.to_csv('df_encode3.csv',index=False)
df_encode4.to_csv('df_encode4.csv',index=False)


In [246]:
unioned_df_encode = pd.concat([df_encode, df_encode1, df_encode2,df_encode3,df_encode4], ignore_index=True)

unioned_df_encode

,Variable Name,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation
0,Visit,object,None,"{'PRN': 70, '4 years': 36, 'Baseline': 63, '6 ...",KNN Imputation
1,_22Jul2009VisitType,object,None,"{'nan': 4, 'Initial Screening Visit': 1, 'Resc...",KNN Imputation
2,_15Aug2011VisitType,object,None,"{'nan': 5, 'Rescreen/Confirmatory Screening Vi...",KNN Imputation
3,StorageOfResidualsDuringTrialN,object,None,"{'nan': 1, 'Not applicable': 0, 'no': 2, 'yes'...",KNN Imputation
4,_15Aug2011ConsentForResidualSa,object,None,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",KNN Imputation
...,...,...,...,...,...
617,SeatedArmBloodPressureSystolic,float64,None,None,Use missing indicator
618,SeatedArmBloodPressureDiastoli,float64,None,None,Use missing indicator
619,TemperatureDegreeCelcius,float64,None,None,Use missing indicator
620,HeartRateBPM,float64,None,None,Use missing indicator


In [248]:
unioned_df_encode['Variable Name']

0                               Visit
1                 _22Jul2009VisitType
2                 _15Aug2011VisitType
3      StorageOfResidualsDuringTrialN
4      _15Aug2011ConsentForResidualSa
                    ...              
617    SeatedArmBloodPressureSystolic
618    SeatedArmBloodPressureDiastoli
619          TemperatureDegreeCelcius
620                      HeartRateBPM
621                RespiratoryRateBPM
Name: Variable Name, Length: 622, dtype: object

In [271]:
cl=['MaskID', 'Visit', 'Status', 'GAD65H', 'AgeOfRelativeWithT1D1_1', 'hyperglycemia', 'DegreeOfRelative_1st', 'AgeOfRelativeWithT1D2_1', 'AgeOfRelativeWithT1D3_1', 'Race_AmericanIndianorAlaskaNat', 'AddisonsDiseaseWithinLast6Months', 'AgeOfRelativeWithT1D4_1', 'ProtocolVersion', 'FirstHearAboutTrialN_ExistingTri', 'FirstHearAboutTrialN_Brochure', 'FirstHearAboutTrialN_Poster', 'FirstHearAboutTrialN_RadioTV', 'SeatedArmBPSystolic', 'ScreeningSiteNumber', 'HighcholesterolEver', 'LimitedLifeExpectancyOrImmunos', 'AsthmaWithinLast6Months', 'IsHeightMeasured', 'HeightNotDone', 'Meglitinides', 'Hastheparticipanthadknownconta', 'selecttestkittype', 'EvidenceOfTuberculosisInfectio', 'Hastheparticipanthadknownconta2', 'participanthaveautoantibodies', 'UseImmunosuppressiveAgents', 'Hastheparticipantbeentestedfor', 'ComplicatedMedIssuesOrAbnormal', 'Hastheparticipanthadalossreduc', 'ReceivedInfluenzaVaccination', 'Abdomen', 'AbdomenFindings', 'Allergies', 'AntibodySampleCollected', 'HospitalizationsForHighGlucose', 'AreThere3GlucoseValuesAvailabl', 'InsulinPump', 'Asthma', 'Bilirubin', 'Blood', 'BloodForCellsSampleCollected', 'BllodForDNASampleCollected', 'BloodForRNASampleCollected', 'InflammatoryBowelDiseaseEver', 'InflammatoryBowelDiseaseWithin', 'Glucose', 'HbA1c', 'ICA', 'CD4P', 'CD4CD8', 'ColitisOrColonProblemsEver', 'CongenitalHeartDiseaseEver', 'InsulinRoutineDaily', 'NumOfRelativesWithT1D', '_15Aug2011ConsentForResidualSa', 'ConsenToContinuedStorageOfBloo', 'BloodSamplesFutureTestingPermi', 'PermissionToControlIfTheyTestA', 'ParticipateInDPT1', 'AnyProblemsRecievingStudyDrugt', 'ParticipantUsedImmunosuppresiv', 'systemicsteroids', 'ParticipantHasOneRelativeWithT', 'ParticipantHasKnownSevereActiv', 'ReproductiveChildbearingPotent', 'Doestheparticipanthavereproduc', 'Eczema', 'EosinophilsResultAbormalClinic', 'EpilepsyConvulsionsSeizuresEve', 'InfusionRelatedEventAnaphylaxi', 'DizyLightHeadedPresent', 'FatigueMalaisePresent', 'HeadcahePresent', 'HypertensionPresent', 'HypotensionPresent', 'IVinfiltrationPresent', 'NauseeVomitPresent', 'InfusionRelatedEventOther', 'SkinRashHypersentivityPresent', 'OtherInfections', 'Genitalia', 'GenitaliaFindings', 'AllergiesWithinLastYear', 'CongenitalHeartDiseaseWithinLa', 'EpilepsyConvulsionsSeizuresWit', 'AddisonsDiseaseEver', 'AddisonsDiseaseWithinLastYear', 'AllergiesEver', 'AlopeciaWithinLastYear', 'AlopeciaEver', 'AsthmaWithinLastYear', 'CancerWithinLastYear', 'CeliacDiseaseEver', 'CeliacDiseaseWithinLastYear', 'ColitisOrColonProblemsWithinLa', 'HepatitisOrLiverDiseaseWithinL', 'HighBPwithinLastYear', 'InfectiousMononucleosisWithinL', 'MultipleSclerosisEver', 'MultipleSclerosisWithinLastYea', 'OtherAutoimmuneDiseasesEver', 'OtherAutoimmuneDiseasesWithinL', 'PerniciousAnemiaWithinLastYear', 'PsoriasisWithinLastYear', 'PsoriasisEver', 'RheumatologicEver', 'RheumatologicWithinLastYear', 'ThyroidDiseaseEver', 'ThyroidDiseaseWithinLastYear', 'UlcerWithinLastYear', 'VitiligoWithinLastYear', 'VitiligoEver', 'CancerEver', 'HospitalizedOtherthanDiabetes', 'SelectStudyForPreventionOfT1D', 'ParticipantUsedInsulinOrOtherG', 'ExacerbationOfCRS', 'FebrileIllness10Days', 'OtherChronicDiseases', 'HbA1cSamplesCollected', 'Heart', 'HeartFindings', 'HEENT', 'HeentFindings', 'HematocritResultAbormalClinica', 'HemoglobinResultAbormalClinica', 'HepatitisOrLiverDiseaseEver', 'HighBPever', 'HLASamplesCollected', 'CommunicationWhenNegAutoantibo', 'InfectiousMononucleosisEver', 'EligibileToBeScreenedForTheStu', 'Gender', 'InsulDoseInfoAvailableFor3Days', 'ParticipantUsingImmunosuppresi', 'ParticipantPregnant', 'PotassiumDepletingDiurectics', 'BetaBlockers', 'Corticosteroids', 'Diphenylhydantoin', 'ImmunosuppressiveTherapies', 'Niacin', 'OtherGlucoseLoweringAgents', 'Thiazolidinediones', 'SubjectUsingCGMS', 'Ketones', 'Leukocytes', 'LotNumber1_1', 'LotNumber2_1', 'LotNumber3_1', 'Lungs', 'Lymphatics', 'LymphaticsFindings', 'LymphocytesResultAbormalClinic', 'MCHResultAbormalClinicallySign', 'MCVResultAbormalClinicallySign', 'MedicationPackNumber1_1', 'MIAA', 'MonocytesResultAbormalClinical', 'Musculoskeletal', 'MusculoskeletalFindings', 'NeckThyroid', 'NeckThyroidFindings', 'Neurologic', 'NeurologicFindings', 'NeutrophilsResultAbormalClinic', 'Nitrite', 'OGTTSampleCollected', 'OnlineScreeningReferralSource', 'Other', 'OtherFindings', 'Ethnicity', 'TestingPlaceForAutoAntibodyOut', 'PermissionForHLAgeneticTesting', 'PerniciousAnemiaEver', 'PlasmaSampleCollected', 'ParticipantLocation', 'Protein', 'Pulses', 'RBCResultAbormalClinicallySign', 'RelativeWithT1D1_1', 'RelativeWithT1D2_1', 'RelativeWithT1D3_1', 'RelativeWithT1D4_1', 'RelativeWithT1D5_1', 'SerumMechanisticSerumSampleCol', 'SerumForProteomicsSampleCollec', 'WholeBloodPBMCPlasmaSampleColl', 'SexOfRelativeWithT1D1_1', 'SexOfRelativeWithT1D2_1', 'SexOfRelativeWithT1D3_1', 'SexOfRelativeWithT1D4_1', 'SexOfRelativeWithT1D5_1', 'SexOfRelativeWithT1D6_1', 'SexOfRelativeWithT1D7_1', 'SexOfRelativeWithT1D8_1', 'Skin', 'SkinFindings', 'Specimencollectionkitshipmentd', 'STIstheparticipantcurrentlytak', 'STWastheparticipantcontactedat', 'STHavetherebeenanychangesincon', 'PermissionForSampleStorage', 'BloodTestForFindOut', 'UlcerEver', 'Urobilinogen', 'UrinePregnancyTest', 'Wasaurinepregnancytestcomplete', 'AcetaminophenGivenToParticipan', 'StudyDrugReturned', 'FullDoseInfused', 'AdmittedHospitalDiagnosis', 'UrinePregnancyTestPositive', 'IGRATestPositive', 'Wasthetestresultpositive', 'WBCResultAbormalClinicallySign', 'InformedTestResultWithin', 'RequiredSpecimensCollected', 'OtherProblems', 'InitialScreeningLocation', 'InitialScreeningLocationOtherS', 'WhichProtocolVersion', 'shipspecimencollectionkit', 'Visit_Dt', 'DateResultsReportedByLabUnknow', 'RBCResultsNormal', 'HemoglobinResultsNormal', 'HematocritResultsNormal', 'MCVresultsNormal', 'PlateletCountResultsNormal2', 'MCHresultsNormal', 'MCHCresultsNormal', 'WBCresultsNormal', 'NeutrophilsResultsNormal', 'LymphocytesResultsNormal', 'MonocytesResultsNormal', 'EosinophilsResultsNormal', 'BasophilsResultsNormal', 'Hastheparticipantbeentoahealth', 'Hastheparticipanthadafever', 'Hastheparticipanthadchillsasso', 'Hastheparticipanthadacough', 'Hastheparticipanthadanydifficu', 'Hastheparticipanthadanypersist', 'Hastheparticipantexperiencedvo', '_8Hastheparticipantbeendiagnos', 'Visit_Type', 'Age', 'RelativeWithT1D', 'DegreeOfRelative_2ndor3rd', 'HaveDiabetes', 'Sex', 'Race_Asian', 'Race_BlackorAfricanAmerican', 'Race_NativeHawaiianorOtherPaci', 'Race_Refused', 'Race_Unknownornotreported', 'Race_White', 'Relationship', 
    'FirstHearAboutTrialN_FamilyFrien', 'FirstHearAboutTrialN_Healthcarep',
    'FirstHearAboutTrialN_Tollfreepho', 'FirstHearAboutTrialN_NewspaperMa', 'FirstHearAboutTrialN_Camp', 'FirstHearAboutTrialN_MeetingPres', 'FirstHearAboutTrialN_Internet', 'FirstHearAboutTrialN_Other', 'clinicalcenternumber', 'ScreeningSiteName', 'AgeOfOnsetInRelativeWithT1D1_1', 'AgeRelativeWithT1DStartedIn1_1', 'AgeOfOnsetInRelativeWithT1D2_1', 'AgeRelativeWithT1DStartedIn2_1', 'AgeOfOnsetInRelativeWithT1D3_1', 'AgeRelativeWithT1DStartedIn3_1', 'AgeOfOnsetInRelativeWithT1D4_1', 'AgeRelativeWithT1DStartedIn4_1', 'RelativeWithT1D6_1', 'RelativeWithT1D7_1', 'ParticipantDiagnosedWithT1D', 'AsthmaEver', 'SeatedArmBPDiastolic', 'EverParticipatedInPreventionSt', 'IVGTTSampleCollected', 'IsWeightMeasured', 'SeatedHeartRate', 'SeatedRespiratoryRate', 'CeliacDiseaseWithinLast6Months', 'VitiligoWithinLast6Months', 'ThyroidDiseaseWithinLast6Months', 'PsoriasisWithinLast6Months', 'AlopeciaWithinLast6Months', 'RheumatologicWithinLast6Months', 'AllergiesWithinLast6Months', 'CancerWithinLast6Months', 'UlcerWithinLast6Months', 'HighBPwithinLast6Months', 'PerniciousAnemiaWithinLast6Mo', 'MultipleSclerosisWithinLast6Mo', 'immunomodulatory', 'Metformin', 'AlphaGlucosidaseInhibitors', 'participantdiagnosedT1D', 'IsParticipantPregnant',
    'SignedMonitoringConsentProtoco', 'HighCholesterolWithinLast6Months', 'HighCholestrolWithinLast6Months', 'Meglinitides', 'CurrentPhase', 'Date_of_Registration', 'HospitalizedSinceLastVisit', 'HaveHadComplicationsFromYourDi', 'HowIsWeightObtained', 'HowIsHeightObtained', 'AvgRecordedFastingGlucoses_unit', 'AvgAllRecordedGlucoses_unit', 'LowestGlucose_unit', 'HighestGlucose_unit', 'RememberDateOfMostRecentHbA1c', 'RememberValueOfMostRecentHbA1c', 'SeizuresFaintingLowBloodGlucos', 'IfSpecimensNotCollec_Other', 'OGTTCollected', 'MMTTCollected', 'AreYouUsingInjections', 'DiagnosedAutoImmuneDisease', 'PlateletCountNotDone', 'MCHNotDone', 'MCHCNotDone', 'NeutrophilsNotDone', 'LymphocytesNotDone', 'MonocytesNotDone', 'EosinophilsNotDone', 'BasophilsNotDone', 'DateOfRegistration', 'Subject18', 'CollectBloodSamples', 'SexGenderPerson', 'DiabeticKetoacidosisTimeOfDiag', 'PregTestPos', 'FeverPresent', 'MethylprednisoneGivenDoseTimeM', 'DiphenhydramineDoseGiven', 'AnyOtherMedications', 'Color', 'Appearance', 'Rbcnotdone', 'HemoglobinNotDone', 'HematocritNotDone', 'MCVNotDone', 'WBCNotDone', 'bloodpressureDiastolicNotDone', 'WeightNotDone', 'Wasaphysicalexamperformedatthi', 'GivenStudyDrugDoseInClinic', 'SignsOfHypoglycemia', 'StudyDrugDispensed', 'ParticipantReceiveStudyDrug', 'MedicationPackNumber2_1', 'MedicationPackNumber3_1', 'UnusualCircumstances', 'STHavetherebeenanychangesinhea', 'INST-10', 'PEP120', 'GLU60', 'PEP60', 'INST0', 'INST60', 'GLU30', 'PEP30', 'INST30', 'GLU0', 'PEP0', 'INST120', 'GLU-10', 'GLU90', 'PEP-10', 'PEP90', 'GLU120', 'INST90', 'GLU15', 'PEP15', 'PEP240', 'GLU240', 'ZNT8', 'PEP150', 'GLU180', 'IA-2H', 'PEP180', 'GLU150', 'PEP210', 'EBVVL', 'GLU210', 'CD8P', 'CD8A', 'CD4A', 'TPRO', 'TBIL', 'SOD', 'POT', 'GLU', 'GLOIgG', 'DBIL', 'CREA', 'CHL', 'BIC', 'ALT', 'ALP', 'ALB', 'UREA', 'AST', 'CPEP', 'INST', 'ICA512', 'GAD65', 'INS0', 'INS120', 'INS-10', 'INS60', 'INS30', 'INS90', 'Height_cm_units', 'Weight_kg_units', 'Spring2019ConsentDateDate', 'DateOfBloodDrawDate', 'DateReportedDate', 'ScreeningDate', 'ConsentSignedDate', 'DateConsentSignedDate', 'SamplesCollectionDate', 'LastHbA1cDate', 'WrittenInformedConsentDate', 'Type1DiabetesDiagnosisDate', 'FluVaccinationDateDate', 'DateTestsRunDate', 'StudyDrugReturnedDate', 'StudyDrugDispensedDate', 'BirthDate', 'DateOfBirthDate', 'HbA1crecordDateDate', 'DateRecordedDate1', 'DateRecordedDate2', 
    'DateRecordedDate3', 'ParticipantDateDispensedDate1_1', 'ParticipantDateDispensedDate2_1', 
    'ParticipantDateDispensedDate3_1', 'MechanisticSampleCollectionDate', 'DateSignedInformedConsentFormDate', 'DateSignedNEWInformedConsentFoDate', 'InterferonGammaReleaseAssayIGRDate', 'OGTTTestStartTime', 'MMTTTestStartTime', 'TimeOfCollection', 'TimeResultsProcessedByLab', 'clinicalcenternumber_missing_indicator', 'AgeOfRelativeWithT1D2_1_missing_indicator', 'AgeOfOnsetInRelativeWithT1D2_1_missing_indicator', 'AgeRelativeWithT1DStartedIn2_1_missing_indicator', 'AgeOfRelativeWithT1D3_1_missing_indicator', 'AgeOfOnsetInRelativeWithT1D3_1_missing_indicator', 'AgeRelativeWithT1DStartedIn3_1_missing_indicator', 'AgeOfRelativeWithT1D4_1_missing_indicator', 'AgeOfOnsetInRelativeWithT1D4_1_missing_indicator', 'AgeRelativeWithT1DStartedIn4_1_missing_indicator', 'SeatedArmBPSystolic_missing_indicator', 'SeatedArmBPDiastolic_missing_indicator', 'SeatedHeartRate_missing_indicator', 'SeatedRespiratoryRate_missing_indicator', 'EosinophilsResultsOR', 'Weight_kg_missing_indicator', 'BasophilsResults', 'MCHCresults_missing_indicator', 'EosinophilsResults_missing_indicator', 'InsulinDoseLast24Hrs', 'PlateletCountResultsNormal_missing_indicator', 'AverageUnitsOfItermediateInsul', 'NeutrophilsResults_missing_indicator', 'LymphocytesResultsOR', 'AvgUnitsShortActingInsulin_missing_indicator', 'AgeRelativeWithT1DStartedIn6_1_missing_indicator', 'HematocritResults_missing_indicator', 'Weight_kg', 'AvgRecordedFastingGlucoses_missing_indicator', 'LymphocytesResults', 'AvgAllRecordedGlucoses_missing_indicator', 'LastHbA1cValue', 'LymphocytesResults_missing_indicator', 'RandomizationNumber', 'CapsulesDispensed', 'Height_cm', 'AgeOfRelativeWithT1D5_1_missing_indicator', 'MCHCresults', 'MMTTAmountConsumed', 'MCHresults', 'RandomizationNumber_missing_indicator', 'TotalOfHomeBloodGlucoseOver3Da', 'Height_cm_missing_indicator', 'AgeOfRelativeWithT1D5_1', 'AvgAllRecordedGlucoses', 'AgeRelativeWithT1DStartedIn5_1', 'BasophilsResultsOR', 'MonocytesResultsOR', 'AgeRelativeWithT1DStartedIn5_1_missing_indicator', 'LowestGlucose_missing_indicator', 'TotalOfHBGMonitoringsLessThan6', 'HeightCm', 'LowestGlucose', 'HemoglobinResults', 'CapsulesDispensed_missing_indicator', 'HeightCm_missing_indicator', 'PlateletCountResultsNormal', 
    'AgeOfOnsetInRelativeWithT1D6_1', 'BasophilsResults_missing_indicator', 'HighestGlucose', 'EosinophilsResults', 'NeutrophilsResultsOR', 'AgeOfRelativeWithT1D6_1_missing_indicator', 'AgeRelativeWithT1DStartedIn6_1', 'HemoglobinResults_missing_indicator', 'TotalOfHomeBloodGlucoseOver3Da_missing_indicator', 'HighestGlucose_missing_indicator', 'WeightKg_missing_indicator', 'CapsulesReturned', 'HematocritResults', 'InsulinDoseLast24Hrs_missing_indicator', 'MCVresults', 'MMTTTimeToConsumeBoost', 'WBCresults_missing_indicator', 'MonocytesResults', 'MMTTTimeToConsumeBoost_missing_indicator', 'EosinophilsResultsOR_missing_indicator', 'AvgUnitsShortActingInsulin', 'AvgRecordedFastingGlucoses', 'BasophilsResultsOR_missing_indicator', 'LymphocytesResultsOR_missing_indicator', 'AverageUnitsOfItermediateInsul_missing_indicator', 'WeightKg', 'NeutrophilsResults', 'AgeOfOnsetInRelativeWithT1D5_1_missing_indicator', 'MonocytesResultsOR_missing_indicator', 'MCVresults_missing_indicator', 'Rbcresults_missing_indicator', 'MCHresults_missing_indicator', 'LastHbA1cValue_missing_indicator', 'AgeOfOnsetInRelativeWithT1D5_1', 'MonocytesResults_missing_indicator', 'WBCresults', 'NeutrophilsResultsOR_missing_indicator', 'TotalOfHBGMonitoringsLessThan6_missing_indicator', 'Rbcresults', 'MMTTAmountConsumed_missing_indicator', 'CapsulesReturned_missing_indicator', 'AgeOfRelativeWithT1D6_1', 'AgeOfOnsetInRelativeWithT1D6_1_missing_indicator']

In [272]:
ll=['Participant Mask ID#', 'Visit', 'Status', 'Anti-GAD65 autoantibody by NIDDK Harmonized radiobinding assay; positive >20 DK units', 'Current Age of Relative', 'Hyperglycemia', 'Degree of relative: 1st', 'Current Age of Relative', 'Current Age of Relative', 'Race: American Indian or Alaska Native', 'Addisons Disease Within Last 6 Months', 'Current Age of Relative', 'Protocol Version', 'How did you first hear about TrialNet: Existing TrialNet Site', 'How did you first hear about TrialNet: Brochure', 'How did you first hear about TrialNet: Poster', 'How did you first hear about TrialNet: Radio/TV', 'Collect the following assessments Seated arm blood pressure systolic', 'Collect the following assessments Seated arm blood pressure Diastolic', 'High Cholesterol ever', 'Limited Life Expectancy or immunosuppressive or immunomodulatory therapies', 'Asthma Within Last 6 Months', 'Is height self measured', 'Height Not Done', 'Is the participant taking Meglitinides', '2Has the participant had known contact with anyone with confirmed COVID19 since', '2If yes please select test kit type', '3 Subject has evidence of prior or current tuberculosis infection as assessed by', '3Has the participant had known contact with anyone with suspected COVID19 since', '4 Has the participant been found to have autoantibodies   outside of TrialNet', '4 Subject requires use of other immunosuppressive agents including chronic use of', '4Has the participant been tested for COVID19 since their previous study visitcontact', '6 Subject has any complicating medical issues or abnormal clinical laboratory', '6Has the participant had a lossreduction of their sense of taste andor smell', '7 Subject has received killed influenza vaccination at least 2 weeks prior to', 'Abdomen', 'Abdomen Findings', 'Allergies', 'Antibodies sample collected', 'Any emergency room visits or hospitalizations for high blood glucose or DKA', 'Are there atleast three glucose values available for atleast three days', 'Are you using an insulin pump', 'Asthma', 'Bilirubin', 'Blood eryuL', 'Blood for cells sample collected', 'Blood for DNA sample collected', 'Blood for RNA sample collected', 'bowel disease ever', 'bowel disease within the last year', 'GlucosemgdL', 'test name', 'test name', 'test name', 'test name', 'Colitis or colon problems ever', 'Congenital heart disease or heart problems ever', 'Daily insulin routines check one', 'deceased relatives', 'Did participant consent to continued storage of residual blood samples in the NIDDK', 'Did participant consent to continued storage of residual blood samples in the NIDDK', 'Did participant give permission for remaining blood samples to be stores for', 'Did participant give permission on the informed consent form to be a control if', 'Did the participant participate in DPT1', 'Did the subject experience any problems since receiving study drug  today or since', 'Does the participant currently use immunosuppressive or immunomodulatory therapies', 'Does the participant currently use immunosuppressive or immunomodulatory therapies', 'Does the participant have at least one blood relative living or deceased with type 1', 'Does the participant have known severe inactive diseases andor diseases which', 'Does the participant have reproductive or childbearing potential', 'Does the participant have reproductive or childbearing potential', 'Eczema', 'Eosinophils results abnormal clinically significant', 'Epilepsy convulsions or seizures ever', 'event Anaphylaxis', 'Event dizzylight headed', 'Event Fatiguemalaise', 'Event Headache', 'Event Hypertension', 'Event Hypotension', 'event IV infiltration', 'Event Nauseavomiting', 'event Other', 'Event Skin RashHypersensitivity', 'Frequent other infections', 'Genitalia', 'Genitalia Findings', 'Has the participant been told by a physician that they he/she has  allergies', 'Has the participant been told by a physician that they he/she has  congenital heart', 'Has the participant been told by a physician that they he/she has  epilepsy convulsions', 'Has the participant been told by a physician that they he/she has addisons disease', 'Has the participant been told by a physician that they he/she has Addisons disease', 'Has the participant been told by a physician that they he/she has allergies ever', 'Has the participant been told by a physician that they he/she has alopecia', 'Has the participant been told by a physician that they he/she has alopecia ever', 'Has the participant been told by a physician that they he/she has asthma', 'Has the participant been told by a physician that they he/she has cancer', 'Has the participant been told by a physician that they he/she has celiac disease', 'Has the participant been told by a physician that they he/she has celiac disease', 'Has the participant been told by a physician that they he/she has colitis or colon', 'Has the participant been told by a physician that they he/she has hepatitis or liver', 'Has the participant been told by a physician that they he/she has high blood pressure', 'Has the participant been told by a physician that they he/she has infectious', 'Has the participant been told by a physician that they he/she has multiple sclerosis', 'Has the participant been told by a physician that they he/she has multiple sclerosis', 'Has the participant been told by a physician that they he/she has other autoimmune', 'Has the participant been told by a physician that they he/she has other autoimmune', 'Has the participant been told by a physician that they he/she has pernicious anemia', 'Has the participant been told by a physician that they he/she has psoriasis', 'Has the participant been told by a physician that they he/she has psoriasis ever', 'Has the participant been told by a physician that they he/she has rheumatologic', 'Has the participant been told by a physician that they he/she has rheumatologic', 'Has the participant been told by a physician that they he/she has thyroid disease', 'Has the participant been told by a physician that they he/she has thyroid disease', 'Has the participant been told by a physician that they he/she has ulcer', 'Has the participant been told by a physician that they he/she has vitiligo', 'Has the participant been told by a physician that they he/she has vitiligo ever', 'Has the participant ever be', 'Has the participant ever been hospitalized other than for diabetes', 'Has the participant ever participated in an observational or prevention study for T1D If', 'Has the participant ever previously used or is currently using medications for the', 'Has the participant experienced exacerbation of Cytokine Release Syndrome CRS', 'Has the subject had a febrile illness within the last ten days', 'Have you been diagnosed with any new chronic diseases other than diabetes', 'HbA1c sample collected', 'Heart', 'Heart Findings', 'HEENT', 'HEENT Findings', 'Hematocrit results abnormal clinically significant', 'Hemoglobin results abnormal clinically significant', 'Hepatitis Liver disease ever', 'High blood pressure ever', 'HLA sample collected', 'If you are negative for autoantibodies we will inform you via', 'Infectious mononucleosis ever', 'Is participant eligible to be screened for the study', 'Is subject male or female', 'Is the insulin dose information available for atleast three days', 'Is the participant currently using immunosuppressive or immunomodulatory therapies', 'Is the participant knowingly pregnant', 'Is the participant taking  potassium depleting diuretics', 'Is the participant taking beta blockers', 'Is the participant taking corticosteroids', 'Is the participant taking diphenylhydantoin', 'Is the participant taking immunosuppressive therapies', 'Is the participant taking niacin', 'Is the participant taking other glucose lowering agents', 'Is the participant taking thiazolidinediones', 'Is the person using a Continuous Glucose Monitoring System CGMS', 'Ketones', 'Leukocytes', 'Lot Number', 'Lot Number', 'Lot Number', 'Lungs', 'Lymphatics', 'Lymphatics Findings', 'Lymphocytes results abnormal clinically significant', 'MCH results abnormal clinically significant', 'MCV results abnormal clinically significant', 'Medication Pack Number', 'Micro Insulin autoantibody by radiobinding assay; positive >0.010 (index)', 'Monocytes results abnormal clinically significant', 'Musculoskeletal', 'Musculoskeletal Findings', 'NeckThyroid', 'NeckThyroid Findings', 'Neurologic', 'Neurologic Findings', 'Neutrophils results abnormal clinically significant', 'Nitrite', 'OGTT sample collected', 'Online screening referral site', 'Other', 'Other Findings', 'Participant Ethnicity', 'Participant get autoantibody testing outside Trialnet', 'Permission for his/her blood to be tested for HLA genes', 'Pernicious anemia ever', 'Plasma Sample Collected', 'Please indicate participant location', 'ProteinmgdL', 'Pulses', 'RBC results abnormal clinically significant', 'Relative with Type 1 Diabetes', 'Relative with Type 1 Diabetes', 'Relative with Type 1 Diabetes', 'Relative with Type 1 Diabetes', 'Relative with Type 1 Diabetes', 'Sample collected for serum mechanistic', 'Sample collected for serum proteomics', 'Sample collected for whole blood PBMC plasma', 'Sex of Relative', 'Sex of Relative', 'Sex of Relative', 'Sex of Relative', 'Sex of Relative', 'Sex of Relative', 'Sex of Relative', 'Sex of Relative', 'Skin', 'Skin Findings', 'Specimen collection kit shipment date', 'ST Is the participant currently taking study drug', 'ST Was the participant contacted at a minimum of every other week between visits', 'STHave there been any changes in concomitant medications since the last', 'Storage of his/her samples for future testing', 'This blood test is being done to find out if', 'Ulcer stomach or duodenal  ever',
    'UrobilinogenmgdL', 'Was a urine pregnancy test completed at this visit', 
    'Was a urine pregnancy test completed at this visit', 'Was Acetaminophen given to the participant',
    'Was study drug returned', 'Was the full dose infused',
    'Was the participant admitted to a hospital during the diagnosis period', 'Was the test result positive', 
    'Was the test result positive', 'Was the test result positive', 'WBC results abnormal clinically significant', 
    'We will let you know your test results within a duration of your blood draw', 
    'Were all required specimens collected for this visit', 'Were there any other problems during the observation period', 
    'Where did initial screening blood draw occur', 'Where did initial screening blood draw occur other specify', 
    'Which version of the Natural History protocol is your site using', 
    'Would you like the TrialNet Coordinating Center to ship an autoantibody', 'Date of visit', 
    'Date results reported by lab Unknown', 'RBC results normal', 'Hemoglobin Results Normal', 'Hematocrit results normal', 
    'MCV results normal', 'Platelet count results normal', 'MCH results normal', 'MCHC results', 'MCHC results normal', 
    'Neutrophils results normal', 'Lymphocytes results normal', 'Monocytes results normal', 'Eosinophils results normal', 
    'Basophils results normal', '1Has the participant been to a health clinic doctors office andor hospital for a sick visit', 
    'C1Has the participant had a fever', '2Has the participant had chills associated with fever', 
    '3Has the participant had a cough', '4Has the participant had any difficulty breathing or shortness of breath', 
    '5Has the participant had any persistent or prolonged fatigue',
    '6Has the participant had a lossreduction of their sense of taste andor smell', 
    '8Has the participant been diagnosed with pneumonia', 'Visit Type', 'Age', 'Do you have a relative with T1D', 
    'Degree of relative: 2nd or 3rd', 'Do you have diabetes', 'Participant Sex', 'Race: Asian', 'Race: Black or African American', 
    'Race: Native Hawaiian or Other Pacific Islander', 'Race: Refused', 'Race: Unknown or not reported', 'Race: White', 
    'Relationship to participant', 'How did you first hear about TrialNet: Family/Friend', 
    'How did you first hear about TrialNet: Health care professional', 
    'How did you first hear about TrialNet: Toll free phone number', 
    'How did you first hear about TrialNet: Newspaper/Magazine', 'How did you first hear about TrialNet: Camp', 
    'How did you first hear about TrialNet: Meeting/Presentation', 'How did you first hear about TrialNet: Internet', 
    'How did you first hear about TrialNet: Other', 'Regional clinical center number', 'Screening site name', 
    'Age of Onset in Relative', 'Age Relative Started Insulin', 'Age of Onset in Relative', 'Age Relative Started Insulin', 
    'Age of Onset in Relative', 'Age Relative Started Insulin', 'Age of Onset in Relative', 'Age Relative Started Insulin', 
    'Relative with type 1 diabetes', 'Relative with type 1 diabetes', 
    'Has the participant been diagnosed with type 1 diabetes T1D', 'Asthma ever', 'Diastolic seated arm blood pressure', 
    'Has the participant ever participated in a prevention study for T1D', 'IVGTT sample collected', 'Is weight measured', 
    'Collect the following assessments Seated heart rate', 'Collect the following assessments Seated respiratory rate', 
    'Celiac Disease Within Last 6 Months', 'Vitiligo Within Last 6 Months', 'Thyroid Disease Within Last 6 Months', 
    'Psoriasis Within Last 6 Months', 'Alopecia Within Last 6 Months', 'Rheumatologic Within Last 6 Months', 
    'Allergies Within Last 6 Months', 'Cancer Within Last 6 Months', 'Ulcer Within Last 6 Months', 'High BP Within Last 6 Months', 
    'Pernicious Anemia Within Last 6 Month', 'Multiple Sclerosis Within Last 6 Months', 'immunomodulatory',
    'Is the participant taking metformin', 'Is the participant taking alpha glucosidase inhibitors',
    'Has the participant been diagnosed with type 1 diabetes (T1D)', 'Is the participant pregnant planning to become pregnant', 'Did the participant previously sign a monitoring consent form for protocol version 15Aug11', 'High Cholesterol Within Last 6 Months', 'High Cholestrol Within Last 6 Months', 'Is the participant taking meglinitides', 'On which study phase is this participant currently enrolled', 'Date of Registration', 'Have you been hospitalized for any reason since your last visitcontact', 'Have you ever been told that you have had complications from your diabetes that have affected the following areas', 'How is weight obtained', 'How is height obtained', 'Average of recorded fasting glucoses over three days units', 'Average of all recorded glucoses over three days units', 'Lowest recorded glucose over three days units', 'Highest recorded glucose over three days units', 'Do you remember the date of the most recent HbA1c', 'Do you remember the most recent HbA1c value', 'Any episodes of seizures or loss of consciousness from low blood glucose', 'IfSpecimensNotCollec: Other', 'Was OGTT collected', 'Was MMTT collected', 'AreYouUsingInjections', 'a Has the participant ever been diagnosed with an autoimmune diseases', 'Platelet count Not Done', 'MCH count Not Done', 'MCHC count Not Done', 'Neutrophils Not Done', 'Lymphocytes Not Done', 'Monocytes Not Done', 'Eosinophils Not Done', 'Basophils Not Done', 'Date of Registration', 'a Is subject 18 years of age', 'a Did the participant give permission to put any remaining blood samples including genetic samples in the NIDDK repository', 'Sex', 'Did the participant have Diabetic Ketoacidosis DKA at time of diagnosis', 'If FEMALE with reproductive or childbearing potential was the pregnancy test result positive', 'Event Fever', 'Was Methylprednisone given to the participant', 'Was Diphenhydramine given to the participant', 'Were any other medications given', 'Color', 'Appearance', 'RBC Not Done', 'Hemoglobin Not Done', 'Hematocrit Not Done', 'MCV Not Done', 'WBC Not Done', 'blood pressure Diastolic Not Done', 'Weight Not Done', 'Was a physical exam performed at this visit', 'Was the participant given their study drug dose in clinic', 'Were there any signs or symptoms of hypoglycemia during the observation period', 'Was study drug dispensed', 'How did the participant receive the study drug', 'Medication Pack Number', 'Medication Pack Number', 'Were there any unusual circumstances', 'ST Have there been any changes in health since the last scheduled visit', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'test name', 'unit of height', 'unit of weight', 'Consent Date 2019 Protocol Version', 'Date of blood draw', 'Date results reported by lab', 'Screening date', 'Informed consent signed date', 'Date signed informed consent form', 'Samples collection date', 'Last HbA1c date', '1 Date written informed consentassent was obtained', 'Date of diagnosis of type 1 diabetes', 'Date of Flu Vaccination', 'Date tests run', 'study drug returned date', 'Study drug dispensed date', 'Participant Birth month and year', 'Date of Birth Month and Year', 'If known record date HbA1c was measured', 'Dates glucoseinsulin recorded', 'Dates glucoseinsulin recorded', 'Dates glucoseinsulin recorded', 'Study drug dispensed to the participant Date Dispensed', 'Study drug dispensed to the participant Date Dispensed', 'Study drug dispensed to the participant Date Dispensed', 'Mechanistic Sample Collection Date', 'Monitoring informed consent form signed date', 'New informed consent form signed date', 'Date of InterferonGamma Release Assay IGRA test', 'OGTT Start Time of Tolerance Test', 'MMTT Start Time of Tolerance Test', 'Time of Collection', 'Time Results Processed by Lab', 'missing value indicator of clinicalcenternumber', 'missing value indicator of AgeOfRelativeWithT1D2_1', 'missing value indicator of AgeOfOnsetInRelativeWithT1D2_1', 'missing value indicator relative AgeRelativeWithT1DStartedIn2_1', 'missing value indicator of AgeOfRelativeWithT1D3_1', 'missing value indicator of AgeOfOnsetInRelativeWithT1D3_1', 'missing value indicator of AgeRelativeWithT1DStartedIn3_1', 'missing value indicator of AgeOfRelativeWithT1D4_1', 'missing value indicator of AgeOfOnsetInRelativeWithT1D4_1', 'missing value indicator of AgeRelativeWithT1DStartedIn4_1', 'missing value indicator of SeatedArmBPSystolic', 'missing value indicator of SeatedArmBPDiastolic', 'missing value indicator of SeatedHeartRate', 'missing value indicator of SeatedRespiratoryRate', 'Eosinophils results OR', 'missing value indicator of Weight_kg', 'Basophils results', 'missing value indicator of MCHCresults', 'missing value indicator of EosinophilsResults', 'What is your average total insulin dose in 24 hours', 'missing value indicator of PlateletCountResultsNormal', 'Average unitsday of intermediatelong acting insulin average over 3 day period', 'missing value indicator of NeutrophilsResults', 'Lymphocytes results OR', 'missing value indicator of AvgUnitsShortActingInsulin', 'missing value indicator of AgeRelativeWithT1DStartedIn6_1', 'missing value indicator of HematocritResults', 'weight in kg', 'missing value indicator of AvgRecordedFastingGlucoses', 'Lymphocytes results', 'missing value indicator of AvgAllRecordedGlucoses', 'Last HbA1c value', 'missing value indicator of LymphocytesResults', 'Record the Randomization Number used for study drug dispensation', 'Number of capsules dispensed', 'Collect the following assessments Height in cm', 'missing value indicator of AgeOfRelativeWithT1D5_1', 'MCHC results', 'MMTT What was the amount consumed', 'MCH results', 'missing value indicator of RandomizationNumber', 'Total number of home blood glucose monitorings over three days', 'missing value indicator of Height_cm', 'Current Age of Relative', 'Average of all recorded glucoses over three days', 'Age Relative Started Insulin', 'Basophils results OR', 'Monocytes results OR', 'missing value indicator of AgeRelativeWithT1DStartedIn5_1', 'missing value indicator of LowestGlucose', 'Number of home blood glucose monitorings over three days that were less than 65mgdl', 'Height cm', 'Lowest recorded glucose over three days', 'Hemoglobin Results', 'missing value indicator of CapsulesDispensed', 'missing value indicator of HeightCm', 'Platelet count results', 'Age of Onset in Relative', 'missing value indicator of BasophilsResults', 'Highest recorded glucose over three days', 'Eosinophils results', 'Neutrophils results OR', 'missing value indicator of AgeOfRelativeWithT1D6_1', 'Age Relative Started Insulin', 'missing value indicator of HemoglobinResults', 'missing value indicator of TotalOfHomeBloodGlucoseOver3Da', 'missing value indicator of HighestGlucose', 'missing value indicator of WeightKg', 'Number of capsules returned', 'Hematocrit results', 'missing value indicator of InsulinDoseLast24Hrs', 'MCV results', 'MMTT How long did it take for the participant to consume the Boost', 'missing value indicator of WBCresults', 'Monocytes results', 'missing value indicator of MMTTTimeToConsumeBoost', 'missing value indicator of EosinophilsResultsOR', 'Average unitsday of short acting insulin', 'Average of recorded fasting glucoses over three days', 'missing value indicator of BasophilsResultsOR', 'missing value indicator of LymphocytesResultsOR', 'missing value indicator of AverageUnitsOfItermediateInsul', 'WeightKGDouble', 'Neutrophils results', 'missing value indicator of AgeOfOnsetInRelativeWithT1D5_1', 'missing value indicator of MonocytesResultsOR', 'missing value indicator of MCVresults', 'missing value indicator of Rbcresults', 'missing value indicator of MCHresults', 'missing value indicator of LastHbA1cValue', 'Age of Onset in Relative', 'missing value indicator of MonocytesResults', 'WBC results', 'missing value indicator of NeutrophilsResultsOR', 'missing value indicator of TotalOfHBGMonitoringsLessThan6', 'RBC results', 'missing value indicator of MMTTAmountConsumed', 
    'missing value indicator of CapsulesReturned', 
    'Current Age of Relative', 'missing value indicator of AgeOfOnsetInRelativeWithT1D6_1']

In [291]:
df=pd.DataFrame({'Variable Name':cl,'Description':ll})

In [292]:
df

,Variable Name,Description
0,MaskID,Participant Mask ID#
1,Visit,Visit
2,Status,Status
3,GAD65H,Anti-GAD65 autoantibody by NIDDK Harmonized ra...
4,AgeOfRelativeWithT1D1_1,Current Age of Relative
...,...,...
554,Rbcresults,RBC results
555,MMTTAmountConsumed_missing_indicator,missing value indicator of MMTTAmountConsumed
556,CapsulesReturned_missing_indicator,missing value indicator of CapsulesReturned
557,AgeOfRelativeWithT1D6_1,Current Age of Relative


In [293]:
import pandas as pd


result = pd.merge(unioned_df_encode, df, on='Variable Name', how='left')




In [327]:
result

,Variable Name,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation,Description
0,Visit,object,None,"{'PRN': 70, '4 years': 36, 'Baseline': 63, '6 ...",KNN Imputation,Visit
1,_22Jul2009VisitType,object,None,"{'nan': 4, 'Initial Screening Visit': 1, 'Resc...",KNN Imputation,NaN
2,_15Aug2011VisitType,object,None,"{'nan': 5, 'Rescreen/Confirmatory Screening Vi...",KNN Imputation,NaN
3,StorageOfResidualsDuringTrialN,object,None,"{'nan': 1, 'Not applicable': 0, 'no': 2, 'yes'...",KNN Imputation,NaN
4,_15Aug2011ConsentForResidualSa,object,None,"{'nan': 3, 'Yes': 2, 'No': 0, 'Unknown': 1}",KNN Imputation,Did participant consent to continued storage o...
5,WhichProtocolVersion,object,None,"{'nan': 4, '15August2011': 0, '22July2009': 2,...",KNN Imputation,Which version of the Natural History protocol ...
6,WhyAccessingParticipantRecord,object,None,{'To enter or view data from a visit which occ...,KNN Imputation,NaN
7,ProtocolVersion,object,None,"{'nan': 7, '3.0': 2, '4.0': 3, '2.0': 1, 'Octo...",KNN Imputation,Protocol Version
8,WhichProtocolVersion2,object,None,"{'nan': 2, '22February2019': 0, 'October 10, 2...",KNN Imputation,NaN
9,AETerm,object,None,"{'nan': 130, 'Pancreatic endocrine: glucose i...",KNN Imputation,NaN


In [329]:
rows_with_nan_in_column1 = result[result['Description'].isna()]
len(rows_with_nan_in_column1)

177

In [299]:
VariableName=['Visit_Dt', 'DateResultsReportedByLabUnknow', 'RBCResultsNormal', 'HemoglobinResultsNormal', 'HematocritResultsNormal', 'MCVresultsNormal', 'PlateletCountResultsNormal2', 'MCHresultsNormal', 'MCHCresultsNormal', 'WBCresultsNormal', 'NeutrophilsResultsNormal', 'LymphocytesResultsNormal', 'MonocytesResultsNormal', 'EosinophilsResultsNormal', 'BasophilsResultsNormal', 'Hastheparticipantbeentoahealth', 'Hastheparticipanthadafever', 'Hastheparticipanthadchillsasso', 'Hastheparticipanthadacough', 'Hastheparticipanthadanydifficu', 'Hastheparticipanthadanypersist', 'Hastheparticipantexperiencedvo', '_8Hastheparticipantbeendiagnos', 'Visit_Type', 'Age', 'RelativeWithT1D', 'DegreeOfRelative_2ndor3rd', 'HaveDiabetes', 'Sex', 'Race_Asian', 'Race_BlackorAfricanAmerican', 'Race_NativeHawaiianorOtherPaci', 'Race_Refused', 'Race_Unknownornotreported', 'Race_White', 'Relationship', 'FirstHearAboutTrialN_FamilyFrien', 'FirstHearAboutTrialN_Healthcarep', 'FirstHearAboutTrialN_Tollfreepho', 'FirstHearAboutTrialN_NewspaperMa', 'FirstHearAboutTrialN_Camp', 'FirstHearAboutTrialN_MeetingPres', 'FirstHearAboutTrialN_Internet', 'FirstHearAboutTrialN_Other', 'clinicalcenternumber', 'ScreeningSiteName', 'AgeOfOnsetInRelativeWithT1D1_1', 'AgeRelativeWithT1DStartedIn1_1', 'AgeOfOnsetInRelativeWithT1D2_1', 'AgeRelativeWithT1DStartedIn2_1', 'AgeOfOnsetInRelativeWithT1D3_1', 'AgeRelativeWithT1DStartedIn3_1', 'AgeOfOnsetInRelativeWithT1D4_1', 'AgeRelativeWithT1DStartedIn4_1', 'RelativeWithT1D6_1', 'RelativeWithT1D7_1', 'ParticipantDiagnosedWithT1D', 'AsthmaEver', 'SeatedArmBPDiastolic', 'EverParticipatedInPreventionSt', 'IVGTTSampleCollected', 'IsWeightMeasured', 'SeatedHeartRate', 'SeatedRespiratoryRate', 'CeliacDiseaseWithinLast6Months', 'VitiligoWithinLast6Months', 'ThyroidDiseaseWithinLast6Months', 'PsoriasisWithinLast6Months', 'AlopeciaWithinLast6Months', 'RheumatologicWithinLast6Months', 'AllergiesWithinLast6Months', 'CancerWithinLast6Months', 'UlcerWithinLast6Months', 'HighBPwithinLast6Months', 'PerniciousAnemiaWithinLast6Mo', 'MultipleSclerosisWithinLast6Mo', 'immunomodulatory', 'Metformin', 'AlphaGlucosidaseInhibitors', 'participantdiagnosedT1D', 'IsParticipantPregnant', 'SignedMonitoringConsentProtoco', 'HighCholesterolWithinLast6Months', 'HighCholestrolWithinLast6Months', 'Meglinitides', 'CurrentPhase', 'Date_of_Registration', 'HospitalizedSinceLastVisit', 'HaveHadComplicationsFromYourDi', 'HowIsWeightObtained', 'HowIsHeightObtained', 'AvgRecordedFastingGlucoses_unit', 'AvgAllRecordedGlucoses_unit', 'LowestGlucose_unitl', 'HighestGlucose_unit', 'RememberDateOfMostRecentHbA1c', 'RememberValueOfMostRecentHbA1c', 'SeizuresFaintingLowBloodGlucos', 'IfSpecimensNotCollec_Other', 'OGTTCollected', 'MMTTCollected', 'AreYouUsingInjections', 'DiagnosedAutoImmuneDisease', 'PlateletCountNotDone', 'MCHNotDone', 'MCHCNotDone', 'NeutrophilsNotDone', 'LymphocytesNotDone', 'MonocytesNotDone', 'EosinophilsNotDone', 'BasophilsNotDone', 'DateOfRegistration', 'Subject18', 'CollectBloodSamples', 'SexGenderPerson', 'DiabeticKetoacidosisTimeOfDiag', 'PregTestPos', 'FeverPresent', 'MethylprednisoneGivenDoseTimeM', 'DiphenhydramineDoseGiven', 'AnyOtherMedications', 'Color', 'Appearance', 'Rbcnotdone', 'HemoglobinNotDone', 'HematocritNotDone', 'MCVNotDone', 'WBCNotDone', 'bloodpressureDiastolicNotDone', 'WeightNotDone', 'Wasaphysicalexamperformedatthi', 'GivenStudyDrugDoseInClinic', 'SignsOfHypoglycemia', 'StudyDrugDispensed', 'ParticipantReceiveStudyDrug', 'MedicationPackNumber2_1', 'MedicationPackNumber3_1', 'UnusualCircumstances', 'STHavetherebeenanychangesinhea', 'INST-10', 'PEP120', 'GLU60', 'PEP60', 'INST0', 'INST60', 'GLU30', 'PEP30', 'INST30', 'GLU0', 'PEP0', 'INST120', 'GLU-10', 'GLU90', 'PEP-10', 'PEP90', 'GLU120', 'INST90', 'GLU15', 'PEP15', 'PEP240', 'GLU240', 'ZNT8', 'PEP150', 'GLU180', 'IA-2H', 'PEP180', 'GLU150', 'PEP210', 'EBVVL', 'GLU210', 'CD8P', 'CD8A', 'CD4A', 'TPRO', 'TBIL', 'SOD', 'POT', 'GLU', 'GLOIgG', 'DBIL', 'CREA', 'CHL', 'BIC', 'ALT', 'ALP', 'ALB', 'UREA', 'AST', 'CPEP', 'INST', 'ICA512', 'GAD65', 'INS0', 'INS120', 'INS-10', 'INS60', 'INS30', 'INS90', 'Height_cm_units', 'Weight_kg_units', 'Spring2019ConsentDateDate', 'DateOfBloodDrawDate', 'DateReportedDate', 'ScreeningDate', 'ConsentSignedDate', 'DateConsentSignedDate', 'SamplesCollectionDate', 'LastHbA1cDate', 'WrittenInformedConsentDate', 'Type1DiabetesDiagnosisDate', 'FluVaccinationDateDate', 'DateTestsRunDate', 'StudyDrugReturnedDate', 'StudyDrugDispensedDate', 'BirthDate', 'DateOfBirthDate', 'HbA1crecordDateDate', 'DateRecordedDate1', 'DateRecordedDate2', 'DateRecordedDate3', 'ParticipantDateDispensedDate1_1', 'ParticipantDateDispensedDate2_1', 'ParticipantDateDispensedDate3_1', 'MechanisticSampleCollectionDate', 'DateSignedInformedConsentFormDate', 'DateSignedNEWInformedConsentFoDate', 'InterferonGammaReleaseAssayIGRDate', 'OGTTTestStartTime', 'MMTTTestStartTime', 'TimeOfCollection', 'TimeResultsProcessedByLab', 'clinicalcenternumber_missing_indicator', 'AgeOfRelativeWithT1D2_1_missing_indicator', 'AgeOfOnsetInRelativeWithT1D2_1_missing_indicator', 'AgeRelativeWithT1DStartedIn2_1_missing_indicator', 'AgeOfRelativeWithT1D3_1_missing_indicator', 'AgeOfOnsetInRelativeWithT1D3_1_missing_indicator', 'AgeRelativeWithT1DStartedIn3_1_missing_indicator', 'AgeOfRelativeWithT1D4_1_missing_indicator', 'AgeOfOnsetInRelativeWithT1D4_1_missing_indicator', 'AgeRelativeWithT1DStartedIn4_1_missing_indicator', 'SeatedArmBPSystolic_missing_indicator', 'SeatedArmBPDiastolic_missing_indicator', 'SeatedHeartRate_missing_indicator', 'SeatedRespiratoryRate_missing_indicator', 'EosinophilsResultsOR', 'Weight_kg_missing_indicator', 'BasophilsResults', 'MCHCresults_missing_indicator', 'EosinophilsResults_missing_indicator', 'InsulinDoseLast24Hrs', 'PlateletCountResultsNormal_missing_indicator', 'AverageUnitsOfItermediateInsul', 'NeutrophilsResults_missing_indicator', 'LymphocytesResultsOR', 'AvgUnitsShortActingInsulin_missing_indicator', 'AgeRelativeWithT1DStartedIn6_1_missing_indicator', 'HematocritResults_missing_indicator', 'Weight_kg', 'AvgRecordedFastingGlucoses_missing_indicator', 'LymphocytesResults', 'AvgAllRecordedGlucoses_missing_indicator', 'LastHbA1cValue', 'LymphocytesResults_missing_indicator', 'RandomizationNumber', 'CapsulesDispensed', 'Height_cm', 'AgeOfRelativeWithT1D5_1_missing_indicator', 'MCHCresults', 'MMTTAmountConsumed', 'MCHresults', 'RandomizationNumber_missing_indicator', 'TotalOfHomeBloodGlucoseOver3Da', 'Height_cm_missing_indicator', 'AgeOfRelativeWithT1D5_1', 'AvgAllRecordedGlucoses', 'AgeRelativeWithT1DStartedIn5_1', 'BasophilsResultsOR', 'MonocytesResultsOR', 'AgeRelativeWithT1DStartedIn5_1_missing_indicator', 'LowestGlucose_missing_indicator', 'TotalOfHBGMonitoringsLessThan6', 'HeightCm', 'LowestGlucose', 'HemoglobinResults', 'CapsulesDispensed_missing_indicator', 'HeightCm_missing_indicator', 'PlateletCountResultsNormal', 'AgeOfOnsetInRelativeWithT1D6_1', 'BasophilsResults_missing_indicator', 'HighestGlucose', 'EosinophilsResults', 'NeutrophilsResultsOR', 'AgeOfRelativeWithT1D6_1_missing_indicator', 'AgeRelativeWithT1DStartedIn6_1', 'HemoglobinResults_missing_indicator', 'TotalOfHomeBloodGlucoseOver3Da_missing_indicator', 'HighestGlucose_missing_indicator', 'WeightKg_missing_indicator', 'CapsulesReturned', 'HematocritResults', 'InsulinDoseLast24Hrs_missing_indicator', 'MCVresults', 'MMTTTimeToConsumeBoost', 'WBCresults_missing_indicator', 'MonocytesResults', 'MMTTTimeToConsumeBoost_missing_indicator', 'EosinophilsResultsOR_missing_indicator', 'AvgUnitsShortActingInsulin', 'AvgRecordedFastingGlucoses', 'BasophilsResultsOR_missing_indicator', 'LymphocytesResultsOR_missing_indicator', 'AverageUnitsOfItermediateInsul_missing_indicator', 'WeightKg', 'NeutrophilsResults', 'AgeOfOnsetInRelativeWithT1D5_1_missing_indicator', 'MonocytesResultsOR_missing_indicator', 'MCVresults_missing_indicator', 'Rbcresults_missing_indicator', 'MCHresults_missing_indicator', 'LastHbA1cValue_missing_indicator', 'AgeOfOnsetInRelativeWithT1D5_1', 'MonocytesResults_missing_indicator', 'WBCresults', 'NeutrophilsResultsOR_missing_indicator', 'TotalOfHBGMonitoringsLessThan6_missing_indicator', 'Rbcresults', 'MMTTAmountConsumed_missing_indicator', 'CapsulesReturned_missing_indicator', 'AgeOfRelativeWithT1D6_1', 'AgeOfOnsetInRelativeWithT1D6_1_missing_indicator']
Variable_Manipulation=['No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']
Measurement_Unit=['NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'mg/dL', 'mg/dL', 'NaN', 'mg/dL', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'cm', 'kg', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', '% of WBC', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', '% of WBC', 'NaN', 'NaN', 'NaN', 'kg', 'NaN', '% of WBC', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'cm', 'NaN', 'g/dL', 'NaN', 'pg', 'NaN', 'NaN', 'NaN', 'NaN', 'mg/dL', 'NaN', 'NaN', '% of WBC', 'NaN', 'NaN', 'NaN', 'cm', 'mg/dL', 'mg/dL', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'mg/dL', '% of WBC', '% of WBC', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', '% of blood', 'NaN', 'fL', 'NaN', 'NaN', '% of WBC', 'NaN', 'NaN', 'NaN', 'mg/dL', 'NaN', 'NaN', 'NaN', 'kg', '% of WBC', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'x10^9 cells/L', 'NaN', 'NaN', 'x10^12 cells/L', 'NaN', 'NaN', 'NaN', 'NaN']

In [309]:
VariableName=['Visit_Dt', 'DateResultsReportedByLabUnknow', 'RBCResultsNormal', 'HemoglobinResultsNormal', 'HematocritResultsNormal', 'MCVresultsNormal', 'PlateletCountResultsNormal2', 'MCHresultsNormal', 'MCHCresultsNormal', 'WBCresultsNormal', 'NeutrophilsResultsNormal', 'LymphocytesResultsNormal', 'MonocytesResultsNormal', 'EosinophilsResultsNormal', 'BasophilsResultsNormal', 'Hastheparticipantbeentoahealth', 'Hastheparticipanthadafever', 'Hastheparticipanthadchillsasso', 'Hastheparticipanthadacough', 'Hastheparticipanthadanydifficu', 'Hastheparticipanthadanypersist', 'Hastheparticipantexperiencedvo', '_8Hastheparticipantbeendiagnos', 'Visit_Type', 'Age', 'RelativeWithT1D', 'DegreeOfRelative_2ndor3rd', 'HaveDiabetes', 'Sex', 'Race_Asian', 'Race_BlackorAfricanAmerican', 'Race_NativeHawaiianorOtherPaci', 'Race_Refused', 'Race_Unknownornotreported', 'Race_White', 'Relationship', 'FirstHearAboutTrialN_FamilyFrien', 'FirstHearAboutTrialN_Healthcarep', 'FirstHearAboutTrialN_Tollfreepho', 'FirstHearAboutTrialN_NewspaperMa', 'FirstHearAboutTrialN_Camp', 'FirstHearAboutTrialN_MeetingPres', 'FirstHearAboutTrialN_Internet', 'FirstHearAboutTrialN_Other', 'clinicalcenternumber', 'ScreeningSiteName', 'AgeOfOnsetInRelativeWithT1D1_1', 'AgeRelativeWithT1DStartedIn1_1', 'AgeOfOnsetInRelativeWithT1D2_1', 'AgeRelativeWithT1DStartedIn2_1', 'AgeOfOnsetInRelativeWithT1D3_1', 'AgeRelativeWithT1DStartedIn3_1', 'AgeOfOnsetInRelativeWithT1D4_1', 'AgeRelativeWithT1DStartedIn4_1', 'RelativeWithT1D6_1', 'RelativeWithT1D7_1', 'ParticipantDiagnosedWithT1D', 'AsthmaEver', 'SeatedArmBPDiastolic', 'EverParticipatedInPreventionSt', 'IVGTTSampleCollected', 'IsWeightMeasured', 'SeatedHeartRate', 'SeatedRespiratoryRate', 'CeliacDiseaseWithinLast6Months', 'VitiligoWithinLast6Months', 'ThyroidDiseaseWithinLast6Months', 'PsoriasisWithinLast6Months', 'AlopeciaWithinLast6Months', 'RheumatologicWithinLast6Months', 'AllergiesWithinLast6Months', 'CancerWithinLast6Months', 'UlcerWithinLast6Months', 'HighBPwithinLast6Months', 'PerniciousAnemiaWithinLast6Mo', 'MultipleSclerosisWithinLast6Mo', 'immunomodulatory', 'Metformin', 'AlphaGlucosidaseInhibitors', 'participantdiagnosedT1D', 'IsParticipantPregnant', 'SignedMonitoringConsentProtoco', 'HighCholesterolWithinLast6Months', 'HighCholestrolWithinLast6Months', 'Meglinitides', 'CurrentPhase', 'Date_of_Registration', 'HospitalizedSinceLastVisit', 'HaveHadComplicationsFromYourDi', 'HowIsWeightObtained', 'HowIsHeightObtained', 'AvgRecordedFastingGlucoses_unit', 'AvgAllRecordedGlucoses_unit', 'LowestGlucose_unitl', 'HighestGlucose_unit', 'RememberDateOfMostRecentHbA1c', 'RememberValueOfMostRecentHbA1c', 'SeizuresFaintingLowBloodGlucos', 'IfSpecimensNotCollec_Other', 'OGTTCollected', 'MMTTCollected', 'AreYouUsingInjections', 'DiagnosedAutoImmuneDisease', 'PlateletCountNotDone', 'MCHNotDone', 'MCHCNotDone', 'NeutrophilsNotDone', 'LymphocytesNotDone', 'MonocytesNotDone', 'EosinophilsNotDone', 'BasophilsNotDone', 'DateOfRegistration', 'Subject18', 'CollectBloodSamples', 'SexGenderPerson', 'DiabeticKetoacidosisTimeOfDiag', 'PregTestPos', 'FeverPresent', 'MethylprednisoneGivenDoseTimeM', 'DiphenhydramineDoseGiven', 'AnyOtherMedications', 'Color', 'Appearance', 'Rbcnotdone', 'HemoglobinNotDone', 'HematocritNotDone', 'MCVNotDone', 'WBCNotDone', 'bloodpressureDiastolicNotDone', 'WeightNotDone', 'Wasaphysicalexamperformedatthi', 'GivenStudyDrugDoseInClinic', 'SignsOfHypoglycemia', 'StudyDrugDispensed', 'ParticipantReceiveStudyDrug', 'MedicationPackNumber2_1', 'MedicationPackNumber3_1', 'UnusualCircumstances', 'STHavetherebeenanychangesinhea', 'INST-10', 'PEP120', 'GLU60', 'PEP60', 'INST0', 'INST60', 'GLU30', 'PEP30', 'INST30', 'GLU0', 'PEP0', 'INST120', 'GLU-10', 'GLU90', 'PEP-10', 'PEP90', 'GLU120', 'INST90', 'GLU15', 'PEP15', 'PEP240', 'GLU240', 'ZNT8', 'PEP150', 'GLU180', 'IA-2H', 'PEP180', 'GLU150', 'PEP210', 'EBVVL', 'GLU210', 'CD8P', 'CD8A', 'CD4A', 'TPRO', 'TBIL', 'SOD', 'POT', 'GLU', 'GLOIgG', 'DBIL', 'CREA', 'CHL', 'BIC', 'ALT', 'ALP', 'ALB', 'UREA', 'AST', 'CPEP', 'INST', 'ICA512', 'GAD65', 'INS0', 'INS120', 'INS-10', 'INS60', 'INS30', 'INS90', 'Height_cm_units', 'Weight_kg_units', 'Spring2019ConsentDateDate', 'DateOfBloodDrawDate', 'DateReportedDate', 'ScreeningDate', 'ConsentSignedDate', 'DateConsentSignedDate', 'SamplesCollectionDate', 'LastHbA1cDate', 'WrittenInformedConsentDate', 'Type1DiabetesDiagnosisDate', 'FluVaccinationDateDate', 'DateTestsRunDate', 'StudyDrugReturnedDate', 'StudyDrugDispensedDate', 'BirthDate', 'DateOfBirthDate', 'HbA1crecordDateDate', 'DateRecordedDate1', 'DateRecordedDate2', 'DateRecordedDate3', 'ParticipantDateDispensedDate1_1', 'ParticipantDateDispensedDate2_1', 'ParticipantDateDispensedDate3_1', 'MechanisticSampleCollectionDate', 'DateSignedInformedConsentFormDate', 'DateSignedNEWInformedConsentFoDate', 'InterferonGammaReleaseAssayIGRDate', 'OGTTTestStartTime', 'MMTTTestStartTime', 'TimeOfCollection', 'TimeResultsProcessedByLab', 'clinicalcenternumber_missing_indicator', 'AgeOfRelativeWithT1D2_1_missing_indicator', 'AgeOfOnsetInRelativeWithT1D2_1_missing_indicator', 'AgeRelativeWithT1DStartedIn2_1_missing_indicator', 'AgeOfRelativeWithT1D3_1_missing_indicator', 'AgeOfOnsetInRelativeWithT1D3_1_missing_indicator', 'AgeRelativeWithT1DStartedIn3_1_missing_indicator', 'AgeOfRelativeWithT1D4_1_missing_indicator', 'AgeOfOnsetInRelativeWithT1D4_1_missing_indicator', 'AgeRelativeWithT1DStartedIn4_1_missing_indicator', 'SeatedArmBPSystolic_missing_indicator', 'SeatedArmBPDiastolic_missing_indicator', 'SeatedHeartRate_missing_indicator', 'SeatedRespiratoryRate_missing_indicator', 'EosinophilsResultsOR', 'Weight_kg_missing_indicator', 'BasophilsResults', 'MCHCresults_missing_indicator', 'EosinophilsResults_missing_indicator', 'InsulinDoseLast24Hrs', 'PlateletCountResultsNormal_missing_indicator', 'AverageUnitsOfItermediateInsul', 'NeutrophilsResults_missing_indicator', 'LymphocytesResultsOR', 'AvgUnitsShortActingInsulin_missing_indicator', 'AgeRelativeWithT1DStartedIn6_1_missing_indicator', 'HematocritResults_missing_indicator', 'Weight_kg', 'AvgRecordedFastingGlucoses_missing_indicator', 'LymphocytesResults', 'AvgAllRecordedGlucoses_missing_indicator', 'LastHbA1cValue', 'LymphocytesResults_missing_indicator', 'RandomizationNumber', 'CapsulesDispensed', 'Height_cm', 'AgeOfRelativeWithT1D5_1_missing_indicator', 'MCHCresults', 'MMTTAmountConsumed', 'MCHresults', 'RandomizationNumber_missing_indicator', 'TotalOfHomeBloodGlucoseOver3Da', 'Height_cm_missing_indicator', 'AgeOfRelativeWithT1D5_1', 'AvgAllRecordedGlucoses', 'AgeRelativeWithT1DStartedIn5_1', 'BasophilsResultsOR', 'MonocytesResultsOR', 'AgeRelativeWithT1DStartedIn5_1_missing_indicator', 'LowestGlucose_missing_indicator', 'TotalOfHBGMonitoringsLessThan6', 'HeightCm', 'LowestGlucose', 'HemoglobinResults', 'CapsulesDispensed_missing_indicator', 'HeightCm_missing_indicator', 'PlateletCountResultsNormal', 'AgeOfOnsetInRelativeWithT1D6_1', 'BasophilsResults_missing_indicator', 'HighestGlucose', 'EosinophilsResults', 'NeutrophilsResultsOR', 'AgeOfRelativeWithT1D6_1_missing_indicator', 'AgeRelativeWithT1DStartedIn6_1', 'HemoglobinResults_missing_indicator', 'TotalOfHomeBloodGlucoseOver3Da_missing_indicator', 'HighestGlucose_missing_indicator', 'WeightKg_missing_indicator', 'CapsulesReturned', 'HematocritResults', 'InsulinDoseLast24Hrs_missing_indicator', 'MCVresults', 'MMTTTimeToConsumeBoost', 'WBCresults_missing_indicator', 'MonocytesResults', 'MMTTTimeToConsumeBoost_missing_indicator', 'EosinophilsResultsOR_missing_indicator', 'AvgUnitsShortActingInsulin', 'AvgRecordedFastingGlucoses', 'BasophilsResultsOR_missing_indicator', 'LymphocytesResultsOR_missing_indicator', 'AverageUnitsOfItermediateInsul_missing_indicator', 'WeightKg', 'NeutrophilsResults', 'AgeOfOnsetInRelativeWithT1D5_1_missing_indicator', 'MonocytesResultsOR_missing_indicator', 'MCVresults_missing_indicator', 'Rbcresults_missing_indicator', 'MCHresults_missing_indicator', 'LastHbA1cValue_missing_indicator', 'AgeOfOnsetInRelativeWithT1D5_1', 'MonocytesResults_missing_indicator', 'WBCresults', 'NeutrophilsResultsOR_missing_indicator', 'TotalOfHBGMonitoringsLessThan6_missing_indicator', 'Rbcresults', 'MMTTAmountConsumed_missing_indicator', 'CapsulesReturned_missing_indicator', 'AgeOfRelativeWithT1D6_1', 'AgeOfOnsetInRelativeWithT1D6_1_missing_indicator', 'MaskID', 'Visit', '_15Aug2011ConsentForResidualSa', 'WhichProtocolVersion', 'ProtocolVersion', 'RBCResultAbormalClinicallySign', 'HemoglobinResultAbormalClinica', 'HematocritResultAbormalClinica', 'MCVResultAbormalClinicallySign', 'MCHResultAbormalClinicallySign', 'WBCResultAbormalClinicallySign', 'NeutrophilsResultAbormalClinic', 'LymphocytesResultAbormalClinic', 'MonocytesResultAbormalClinical', 'EosinophilsResultAbormalClinic', 'Hastheparticipanthadknownconta', 'Hastheparticipanthadknownconta2', 'Hastheparticipantbeentestedfor', 'Hastheparticipanthadalossreduc', 'DegreeOfRelative_1st', 'ParticipantLocation', 'Ethnicity', 'Race_AmericanIndianorAlaskaNat', 'FirstHearAboutTrialN_ExistingTri', 'FirstHearAboutTrialN_Brochure', 'FirstHearAboutTrialN_Poster', 'FirstHearAboutTrialN_RadioTV', 'InitialScreeningLocation', 'InitialScreeningLocationOtherS', 'BloodSamplesFutureTestingPermi', 'BloodTestForFindOut', 'CommunicationWhenNegAutoantibo', 'InformedTestResultWithin', 'PermissionToControlIfTheyTestA', 'participanthaveautoantibodies', 'OnlineScreeningReferralSource', 'ScreeningSiteNumber', 'shipspecimencollectionkit', 'Specimencollectionkitshipmentd', 'selecttestkittype', 'NumOfRelativesWithT1D', 'RelativeWithT1D1_1', 'SexOfRelativeWithT1D1_1', 'AgeOfRelativeWithT1D1_1', 'RelativeWithT1D2_1', 'SexOfRelativeWithT1D2_1', 'AgeOfRelativeWithT1D2_1', 'RelativeWithT1D3_1', 'SexOfRelativeWithT1D3_1', 'AgeOfRelativeWithT1D3_1', 'RelativeWithT1D4_1', 'SexOfRelativeWithT1D4_1', 'AgeOfRelativeWithT1D4_1', 'RelativeWithT1D5_1', 'SexOfRelativeWithT1D5_1', 'SexOfRelativeWithT1D6_1', 'SexOfRelativeWithT1D7_1', 'SexOfRelativeWithT1D8_1', 'ParticipantUsedInsulinOrOtherG', 'ParticipantHasKnownSevereActiv', 'ParticipantPregnant', 'PermissionForHLAgeneticTesting', 'PermissionForSampleStorage', 'AsthmaWithinLastYear', 'CeliacDiseaseEver', 'CeliacDiseaseWithinLastYear', 'InflammatoryBowelDiseaseEver', 'InflammatoryBowelDiseaseWithin', 'AddisonsDiseaseEver', 'AddisonsDiseaseWithinLastYear', 'VitiligoEver', 'VitiligoWithinLastYear', 'ThyroidDiseaseEver', 'ThyroidDiseaseWithinLastYear', 'PsoriasisEver', 'PsoriasisWithinLastYear', 'AlopeciaEver', 'AlopeciaWithinLastYear', 'RheumatologicEver', 'RheumatologicWithinLastYear', 'MultipleSclerosisEver', 'MultipleSclerosisWithinLastYea', 'OtherAutoimmuneDiseasesEver', 'OtherAutoimmuneDiseasesWithinL', 'SeatedArmBPSystolic', 'ParticipateInDPT1', 'AllergiesEver', 'AllergiesWithinLastYear', 'AntibodySampleCollected', 'BetaBlockers', 'BllodForDNASampleCollected', 'BloodForCellsSampleCollected', 'BloodForRNASampleCollected', 'CancerEver', 'CancerWithinLastYear', 'ColitisOrColonProblemsEver', 'ColitisOrColonProblemsWithinLa', 'CongenitalHeartDiseaseEver', 'CongenitalHeartDiseaseWithinLa', 'ConsenToContinuedStorageOfBloo', 'Corticosteroids', 'Diphenylhydantoin', 'EpilepsyConvulsionsSeizuresEve', 'EpilepsyConvulsionsSeizuresWit', 'HLASamplesCollected', 'HbA1cSamplesCollected', 'HepatitisOrLiverDiseaseEver', 'HepatitisOrLiverDiseaseWithinL', 'HighBPever', 'HighBPwithinLastYear', 'HighcholesterolEver', 'ImmunosuppressiveTherapies', 'InfectiousMononucleosisEver', 'InfectiousMononucleosisWithinL', 'Niacin', 'OGTTSampleCollected', 'ParticipantHasOneRelativeWithT', 'ParticipantUsedImmunosuppresiv', 'ParticipantUsingImmunosuppresi', 'PerniciousAnemiaEver', 'PerniciousAnemiaWithinLastYear', 'PlasmaSampleCollected', 'PotassiumDepletingDiurectics', 'SerumForProteomicsSampleCollec', 'UlcerEver', 'UlcerWithinLastYear', 'SelectStudyForPreventionOfT1D', 'TestingPlaceForAutoAntibodyOut', 'IsHeightMeasured', 'LimitedLifeExpectancyOrImmunos', 'AsthmaWithinLast6Months', 'AddisonsDiseaseWithinLast6Months', 'hyperglycemia', 'systemicsteroids', 'Thiazolidinediones', 'OtherGlucoseLoweringAgents', 'SerumMechanisticSerumSampleCol', 'WholeBloodPBMCPlasmaSampleColl', 'Meglitinides', 'Status', 'OtherChronicDiseases', 'SubjectUsingCGMS', 'AreThere3GlucoseValuesAvailabl', 'InsulDoseInfoAvailableFor3Days', 'InsulinRoutineDaily', 'HospitalizationsForHighGlucose', 'RequiredSpecimensCollected', 'InsulinPump', 'Asthma', 'Allergies', 'Eczema', 'OtherInfections', 'Other', 'HeentFindings', 'NeckThyroidFindings', 'HeartFindings', 'MusculoskeletalFindings', 'GenitaliaFindings', 'AbdomenFindings', 'LymphaticsFindings', 'SkinFindings', 'NeurologicFindings', 'OtherFindings', 'ReproductiveChildbearingPotent', 'UrinePregnancyTest', 'UrinePregnancyTestPositive', 'ReceivedInfluenzaVaccination', 'EvidenceOfTuberculosisInfectio', 'UseImmunosuppressiveAgents', 'ComplicatedMedIssuesOrAbnormal', 'Gender', 'EligibileToBeScreenedForTheStu', 'HospitalizedOtherthanDiabetes', 'AdmittedHospitalDiagnosis', 'IGRATestPositive', 'FebrileIllness10Days', 'ExacerbationOfCRS', 'SkinRashHypersentivityPresent', 'FatigueMalaisePresent', 'DizyLightHeadedPresent', 'NauseeVomitPresent', 'HypotensionPresent', 'HypertensionPresent', 'HeadcahePresent', 'IVinfiltrationPresent', 'InfusionRelatedEventAnaphylaxi', 'InfusionRelatedEventOther', 'FullDoseInfused', 'AnyProblemsRecievingStudyDrugt', 'AcetaminophenGivenToParticipan', 'Leukocytes', 'Nitrite', 'Protein', 'Glucose', 'Ketones', 'Urobilinogen', 'Bilirubin', 'Blood', 'Doestheparticipanthavereproduc', 'Wasaurinepregnancytestcomplete', 'Wasthetestresultpositive', 'HeightNotDone', 'HEENT', 'NeckThyroid', 'Heart', 'Lungs', 'Pulses', 'Musculoskeletal', 'Genitalia', 'Abdomen', 'Lymphatics', 'Skin', 'Neurologic', 'OtherProblems', 'StudyDrugReturned', 'MedicationPackNumber1_1', 'LotNumber1_1', 'LotNumber2_1', 'LotNumber3_1', 'STHavetherebeenanychangesincon', 'STIstheparticipantcurrentlytak', 'STWastheparticipantcontactedat', 'HbA1c', 'ICA', 'GAD65H', 'MIAA', 'CD4P', 'CD4CD8']



Variable_Manipulation=['No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']




In [310]:
len(VariableName)

559

In [311]:
len(Variable_Manipulation)

559

In [330]:
df=pd.DataFrame({'Variable Name':VariableName,'Variable Manipulation':Variable_Manipulation})

In [331]:
df

,Variable Name,Variable Manipulation
0,Visit_Dt,No
1,DateResultsReportedByLabUnknow,No
2,RBCResultsNormal,Yes
3,HemoglobinResultsNormal,Yes
4,HematocritResultsNormal,Yes
5,MCVresultsNormal,Yes
6,PlateletCountResultsNormal2,Yes
7,MCHresultsNormal,Yes
8,MCHCresultsNormal,Yes
9,WBCresultsNormal,Yes


In [332]:
result22 = pd.merge(result, df, on='Variable Name', how='left')


In [341]:
a=[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'mg/dL', 'mg/dL', nan, 'mg/dL', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'cm', 'kg', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '% of WBC', nan, nan, nan, nan, nan, nan, nan, nan, '% of WBC', nan, nan, nan, 'kg', nan, '% of WBC', nan, nan, nan, nan, nan, 'cm', nan, 'g/dL', nan, 'pg', nan, nan, nan, nan, 'mg/dL', nan, nan, '% of WBC', nan, nan, nan, 'cm', 'mg/dL', nan, nan, nan, nan, nan, nan, 'mg/dL', '% of WBC', '% of WBC', nan, nan, nan, nan, nan, nan, nan, '% of blood', nan, 'fL', nan, nan, '% of WBC', nan, nan, nan, 'mg/dL', nan, nan, nan, 'kg', '% of WBC', nan, nan, nan, nan, nan, nan, nan, nan, 'x10^9 cells/L', nan, nan, 'x10^12 cells/L', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '% of Hb', nan, nan, nan, nan, nan]


NameError: name 'nan' is not defined

In [321]:
pd.set_option('display.max_columns',None)

In [326]:
rows_with_nan_in_column1 = result22[result22['Description'].isna()]
rows_with_nan_in_column1

,Variable Name,Variable Type,Measurement Units as applicable,Corresponding Code List,Missingness Representation,Description,Variable Manipulation
1,_22Jul2009VisitType,object,None,"{'nan': 4, 'Initial Screening Visit': 1, 'Resc...",KNN Imputation,NaN,NaN
2,_15Aug2011VisitType,object,None,"{'nan': 5, 'Rescreen/Confirmatory Screening Vi...",KNN Imputation,NaN,NaN
3,StorageOfResidualsDuringTrialN,object,None,"{'nan': 1, 'Not applicable': 0, 'no': 2, 'yes'...",KNN Imputation,NaN,NaN
6,WhyAccessingParticipantRecord,object,None,{'To enter or view data from a visit which occ...,KNN Imputation,NaN,NaN
8,WhichProtocolVersion2,object,None,"{'nan': 2, '22February2019': 0, 'October 10, 2...",KNN Imputation,NaN,NaN
9,AETerm,object,None,"{'nan': 130, 'Pancreatic endocrine: glucose i...",KNN Imputation,NaN,NaN
10,AECategory,object,None,"{'nan': 38, 'Endocrine': 8, 'Neurology': 26, '...",KNN Imputation,NaN,NaN
11,AETreatLocation,object,None,"{'nan': 5, 'Inpatient': 1, 'Outpatient': 3, 'E...",KNN Imputation,NaN,NaN
12,AECausalityByReporter,object,None,"{'nan': 5, 'Definitely not related': 0, 'Proba...",KNN Imputation,NaN,NaN
13,Eventabateafterstoppingdrug,object,None,"{'nan': 3, 'Not Applicable': 1, 'No': 0, 'Yes'...",KNN Imputation,NaN,NaN


In [342]:
result22.to_csv('data_dictionary.csv',index=False)

In [343]:
rows_with_nan_in_column1 = result[result22['Description'].isna()]
len(rows_with_nan_in_column1)

177

In [344]:
rows_with_nan_in_column1 = result[result22['Variable Manipulation'].isna()]
len(rows_with_nan_in_column1)

178

In [340]:
rows_with_nan_in_column1['Variable Name'].tolist()

['_22Jul2009VisitType',
 '_15Aug2011VisitType',
 'StorageOfResidualsDuringTrialN',
 'WhyAccessingParticipantRecord',
 'WhichProtocolVersion2',
 'AETerm',
 'AECategory',
 'AETreatLocation',
 'AECausalityByReporter',
 'Eventabateafterstoppingdrug',
 'CentralIRBOrlocalIRB',
 'centralIRBscriteria',
 'Followupstatuscode',
 'TN01ParticipantStatusSectionB',
 'Rbcresults_Units',
 'HemoglobinResults_Units',
 'HematocritResults_Units',
 'MCVresults_Units',
 'MCHresults_Units',
 'MCHCresults_Units',
 'WBCresults_Units',
 'NeutrophilsResults_Units',
 'NeutrophilsResultsOR_Units',
 'LymphocytesResults_Units',
 'LymphocytesResultsOR_Units',
 'MonocytesResults_Units',
 'MonocytesResultsOR_Units',
 'EosinophilsResults_Units',
 'EosinophilsResultsOR_Units',
 'BasophilsResults_Units',
 'BasophilsResultsOR_Units',
 'IndicateTheStudyParticipantWas',
 'IsParticipantCurrentlyEligible',
 'IsTheParticipantInterested',
 'Havetherebeenanychangesinthepa',
 'Participantisahotleadwarmleado',
 'TN16TN19TN20Eligibil

In [ ]:
['_22Jul2009VisitType',
 '_15Aug2011VisitType',
 'StorageOfResidualsDuringTrialN',
 'WhyAccessingParticipantRecord',
 'WhichProtocolVersion2',
 'AETerm',
 'AECategory',
 'AETreatLocation',
 'AECausalityByReporter',
 'Eventabateafterstoppingdrug',
 'CentralIRBOrlocalIRB',
 'centralIRBscriteria',
 'Followupstatuscode',
 'TN01ParticipantStatusSectionB',
 'Rbcresults_Units',
 'HemoglobinResults_Units',
 'HematocritResults_Units',
 'MCVresults_Units',
 'MCHresults_Units',
 'MCHCresults_Units',
 'WBCresults_Units',
 'NeutrophilsResults_Units',
 'NeutrophilsResultsOR_Units',
 'LymphocytesResults_Units',
 'LymphocytesResultsOR_Units',
 'MonocytesResults_Units',
 'MonocytesResultsOR_Units',
 'EosinophilsResults_Units',
 'EosinophilsResultsOR_Units',
 'BasophilsResults_Units',
 'BasophilsResultsOR_Units',
 'IndicateTheStudyParticipantWas',
 'IsParticipantCurrentlyEligible',
 'IsTheParticipantInterested',
 'Havetherebeenanychangesinthepa',
 'Participantisahotleadwarmleado',
 'TN16TN19TN20EligibilityStatus',
 'Pleaseselecttypeofcontactattem',
 'FinalContactAttempt1_1',
 'Istheparticipantinterestedinth',
 'Didtheparticipantindicatetheym',
 'Informationgatheredfrom',
 'MethodofContact',
 'ChangeOfStatusOccuring',
 'ReasonForWithdrawal',
 'ParticipantWillingToBeContacte',
 'DiagnosisBy',
 'DiagnosisByOthersSpecify',
 'Polyuria',
 'PolyuriaMonth',
 'Polydipsia',
 'PolydipsiaMonth',
 'Polyphagia',
 'Fatigue',
 'UnexplainedWeightLoss',
 'Ketoacidosis',
 'Units1_1',
 'GlucoseType1_1',
 'MeasuredBy1_1',
 'Units2_1',
 'GlucoseType2_1',
 'MeasuredBy2_1',
 'Units3_1',
 'GlucoseType3_1',
 'MeasuredBy3_1',
 'CriteriaParticipantDiagnosewit',
 'H2OvernightHospitalization',
 'H3AdmissionToICU',
 'H5StartedInsulin',
 'NotificationStatus',
 'ParticipantContacted',
 'ParticipantExpectedToContinue',
 'Old_Site_Number',
 'Current_Site_Number',
 'Reason_for_Transfer',
 'PInotified',
 'ProtocolDeviation',
 'ProtocolDeviationOtherSubcateg',
 'SignedConsentFor2017Protocol',
 'Medication',
 'Units',
 'ConMedsFrequencyChoices',
 'ConMedsFrequencyOtherDetail',
 'ConMedsIntervalChoices',
 'ConMedsRoute',
 'Indication',
 'IsContinuing',
 'LowestGlucose_unit',
 'SelectTheTrialNetStudyInWhichT',
 'SubjectHasSignedWrittenInforme',
 'AllowToPutAnyRemainingBloodSam',
 'TN16ProtocolDeviation',
 'SelectTheVersionOfTN16',
 'aereportdt',
 'AE_Submitted',
 'AEPrimarySecondary',
 'AEexpected',
 'aeresolvedt',
 'AEPatientOutcome',
 'Eventreappearafterreintroduct',
 'AEReasonForFollowup',
 'PhysicalExamFirstVisitYesNo',
 'LungsFindings',
 'PulsesFindings',
 'PubicHairTannerStage',
 'ConMedsIntervalOtherDetails',
 'ExperiencedAnyHypoglycemicEven',
 'VisitProtocolDeviationOccurred',
 'ProtocolDeviationReasons',
 'PINotified',
 'AnyGrade3OrGreaterProblems',
 'GCSFWassplenomegalyindicatedon',
 'SubcutaneousInjectionGiven',
 'ExperienceFollowingProblems',
 'ExperienceOtherProblems',
 'SpecificGravity',
 'PlanPregnantWithinNextYear',
 'BirthControlAbstinenceAcceptab',
 'Event_Title',
 'SampleMaskID',
 'Description',
 'HBsAb',
 'consenttocontinuedstorageofres',
 'WastheVolunteerUnderstandingAs',
 'Hastheparticipantusedantibioti',
 'firstorseconddegreerelativesbe',
 'Hasanyoneintheparticipantfamil',
 'Hastheparticipanteverbeendiagn',
 'DescriptionDeviationCircumstan',
 'Havetherebeenanychangesinhealt',
 'Havetherebeenanychangesinconco',
 'Istheparticipantcurrentlytakin',
 'HbA1c_Units',
 'willingToConsiderAtALaterTime',
 'DateParticipantwascontacted1_2',
 'HospitalizedAtDiagnosisTime',
 'Date_of_Study_Start',
 'Date_of_Diagnosis',
 'aedate',
 'AEFollowUpDate',
 'Date_of_Draw',
 'death_date',
 'ReviewDate',
 'Date_Run',
 'Datewritteninformedconsentasse2',
 '_22Jul2009ConsentSignedDate',
 '_15Aug2011ConsentDateDate',
 'Fall2017ConsentDateDate',
 'DateStatusChangeDate',
 'DoRDate',
 'DoWDate',
 'DiagnosisDate',
 'InsulinTreatmentStartedDate',
 'AdmissionDate',
 'DischargeDate',
 'GlucoseDate1_1',
 'GlucoseDate2_1',
 'GlucoseDate3_1',
 'HbA1cDate',
 'ProtcolDeviationDate',
 'AssessDateDate',
 'StartDateDate',
 'StopDateDate',
 'InformedConsentDate',
 'DateProtocolDeviationDate',
 'ContactDateDate',
 'DateofAssessmentDate',
 'DateCompletionDate',
 'F2ClinicalDiagnosisDateDate',
 'GAD65H_y',
 'CMVVL',
 'Dose',
 'AnthopometricsWeightKg',
 'SeatedArmBloodPressureSystolic',
 'SeatedArmBloodPressureDiastoli',
 'TemperatureDegreeCelcius',
 'HeartRateBPM',
 'RespiratoryRateBPM']

